# Компьютерное зрение: Семинар
## Интерактивная работа с обработкой изображений

**Преподаватель:** Иванов Александр, Т-Банк

**Аудитория:** ФКН ВШЭ, МФТИ

**Дата:** 

На этом семинаре мы разберем практические приемы обработки изображений, используя OpenCV, NumPy и scikit-image.

**Структура семинара:**
- Часть 1: Работа с цветовыми пространствами 
- Часть 2: Фильтрация и морфология 
- Часть 3: Интерполяция и геометрические преобразования
- Часть 4: Поиск паттернов
- Часть 5: Аугментация изображений 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

# Настройка отображения
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12

print("Библиотеки загружены успешно!")
print(f"OpenCV версия: {cv2.__version__}")
print(f"NumPy версия: {np.__version__}")

In [ ]:
# Загружаем тестовое изображение
img_bgr = cv2.imread('test_image_3_1.jpg')
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

# Проверяем размеры и тип
print(f"Размер изображения: {img_rgb.shape}")
print(f"Тип данных: {img_rgb.dtype}")
print(f"Диапазон значений: [{img_rgb.min()}, {img_rgb.max()}]")

# Отображаем оригинал
plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Исходное изображение', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

---

## Часть 1: Работа с цветом и яркостью

### Задание 1.1: Регулировка яркости

**Теория:**

Яркость изображения регулируется путем **добавления константы** ко всем пикселям:

$$
I_{\text{bright}}(x, y) = I(x, y) + \beta
$$

где:
- $I(x, y)$ — исходное значение пикселя
- $\beta$ — сдвиг яркости (может быть положительным или отрицательным)
- Результат должен быть **ограничен диапазоном** [0, 255]

**Важно:**
- $\beta > 0$ → увеличение яркости (изображение светлеет)
- $\beta < 0$ → уменьшение яркости (изображение темнеет)
- Нужно использовать `np.clip()` для ограничения значений

**Задача:**

Реализуйте функцию `adjust_brightness(image, beta)`, которая изменяет яркость изображения.

**Подсказка:**
Что может пойти не так с особенностями реализации uint8?

---

In [ ]:
def adjust_brightness(image: np.ndarray, beta: int) -> np.ndarray:
    """
    Регулировка яркости изображения.
    
    Args:
        image: Входное изображение (RGB)
        beta: Сдвиг яркости (положительный или отрицательный)
    
    Returns:
        Изображение с измененной яркостью
    """
    # ВАШ КОД ЗДЕСЬ
    pass


# Тестирование с тремя значениями beta
beta_values = [-50, 0, 50]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Оригинал
axes[0].imshow(img_rgb)
axes[0].set_title('Оригинал', fontsize=14)
axes[0].axis('off')

# Результаты с разными значениями beta
for i, beta in enumerate(beta_values, start=1):
    result = adjust_brightness(img_rgb, beta)
    axes[i].imshow(result)
    axes[i].set_title(f'β = {beta}', fontsize=14)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

---

### Задание 1.2: Регулировка контрастности

**Теория:**

Контрастность регулируется путем **умножения на коэффициент**:

$$
I_{\text{contrast}}(x, y) = \alpha (\cdot I(x, y) - 128) + 128
$$

где:
- $\alpha$ — коэффициент контрастности
- $\alpha > 1$ → увеличение контрастности (больше разница между светлым и темным)
- $\alpha < 1$ → уменьшение контрастности (меньше разница, изображение "серее")
- $\alpha = 1$ → контрастность не изменяется

**Комбинированная формула:**

Часто яркость и контрастность регулируют вместе:

$$
I_{\text{new}}(x, y) = \alpha \cdot I(x, y) + \beta
$$

**Задача:**

Реализуйте функцию `adjust_contrast(image, alpha)`, которая изменяет контрастность изображения.

**Подсказка:**
Снова аккуратнее с типами!

---

In [ ]:
def adjust_contrast(image: np.ndarray, alpha: float) -> np.ndarray:
    """
    Регулировка контрастности изображения.
    
    Args:
        image: Входное изображение (RGB)
        alpha: Коэффициент контрастности (> 1 увеличивает, < 1 уменьшает)
    
    Returns:
        Изображение с измененной контрастностью
    """
    # ВАШ КОД ЗДЕСЬ
    pass


# Тестирование с тремя значениями alpha
alpha_values = [0.5, 1.0, 1.5]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(img_rgb)
axes[0].set_title('Оригинал', fontsize=14)
axes[0].axis('off')

for i, alpha in enumerate(alpha_values, start=1):
    result = adjust_contrast(img_rgb, alpha)
    axes[i].imshow(result)
    axes[i].set_title(f'α = {alpha}', fontsize=14)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

---

### Задание 1.3: Конвертация в оттенки серого

**Теория:**

Существует несколько способов преобразования RGB в grayscale:

#### 1. Среднее арифметическое (Average)
$$
\text{Gray} = \frac{R + G + B}{3}
$$

**Проблема:** Не учитывает восприятие человеческого глаза.

#### 2. Взвешенное среднее (Luminosity) — **стандартный метод**
$$
\text{Gray} = 0.299 \cdot R + 0.587 \cdot G + 0.114 \cdot B
$$

**Обоснование:**
- Человеческий глаз **наиболее чувствителен к зеленому** цвету (58.7%)
- Менее чувствителен к красному (29.9%)
- Наименее чувствителен к синему (11.4%)
- Эти коэффициенты соответствуют **стандарту ITU-R BT.601**

#### 3. Lightness
$$
\text{Gray} = \frac{\max(R, G, B) + \min(R, G, B)}{2}
$$

**Задача:**

Реализуйте **три функции** преобразования в grayscale и сравните результаты.

**Подсказка:**
Типы-типы-типы!

---

In [ ]:
def rgb_to_gray_average(image: np.ndarray) -> np.ndarray:
    """
    Конвертация RGB в grayscale методом среднего арифметического.
    """
    # ВАШ КОД ЗДЕСЬ
    pass


def rgb_to_gray_luminosity(image: np.ndarray) -> np.ndarray:
    """
    Конвертация RGB в grayscale методом взвешенного среднего (ITU-R BT.601).
    """
    # ВАШ КОД ЗДЕСЬ
    pass


def rgb_to_gray_lightness(image: np.ndarray) -> np.ndarray:
    """
    Конвертация RGB в grayscale методом lightness.
    """
    # ВАШ КОД ЗДЕСЬ
    pass


# Визуализация всех трех методов
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Оригинал
axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Оригинал (RGB)', fontsize=14)
axes[0, 0].axis('off')

# Метод 1: Average
gray_avg = rgb_to_gray_average(img_rgb)
axes[0, 1].imshow(gray_avg, cmap='gray')
axes[0, 1].set_title('Average Method\n(R+G+B)/3', fontsize=14)
axes[0, 1].axis('off')

# Метод 2: Luminosity (стандартный)
gray_lum = rgb_to_gray_luminosity(img_rgb)
axes[1, 0].imshow(gray_lum, cmap='gray')
axes[1, 0].set_title('Luminosity Method (ITU-R BT.601)\n0.299R + 0.587G + 0.114B', fontsize=14)
axes[1, 0].axis('off')

# Метод 3: Lightness
gray_light = rgb_to_gray_lightness(img_rgb)
axes[1, 1].imshow(gray_light, cmap='gray')
axes[1, 1].set_title('Lightness Method\n(max + min) / 2', fontsize=14)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Сравнение с OpenCV
gray_opencv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
print(f"Разница между Luminosity и OpenCV: {np.abs(gray_lum.astype(int) - gray_opencv.astype(int)).max()} (должно быть 0 или ~1)")

---

### Задание 1.4: Регулировка насыщенности

**Теория:**

Насыщенность (Saturation) — это **интенсивность цвета**. 

Для изменения насыщенности нужно:
1. Перейти в цветовое пространство **HSV** (Hue, Saturation, Value)
2. Изменить канал **S** (Saturation)
3. Вернуться обратно в **RGB**

**Цветовое пространство HSV:**
- **H (Hue)** — оттенок (0-179 в OpenCV, 0-360° в градусах)
- **S (Saturation)** — насыщенность (0-255)
- **V (Value)** — яркость (0-255)

**Формула изменения насыщенности:**

$$
S_{\text{new}} = S_{\text{old}} \cdot \text{factor}
$$

где:
- `factor > 1` → увеличение насыщенности (более яркие цвета)
- `factor < 1` → уменьшение насыщенности (ближе к grayscale)
- `factor = 0` → полное обесцвечивание (grayscale)

**Алгоритм:**
```python
1. RGB → HSV: cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
2. Изменить канал S
3. HSV → RGB: cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
```

**Задача:**

Реализуйте функцию `adjust_saturation(image, factor)`, которая изменяет насыщенность изображения.

---

In [ ]:
def adjust_saturation(image: np.ndarray, factor: float) -> np.ndarray:
    """
    Регулировка насыщенности изображения.
    
    Args:
        image: Входное изображение (RGB)
        factor: Коэффициент насыщенности (> 1 увеличивает, < 1 уменьшает)
    
    Returns:
        Изображение с измененной насыщенностью
    """
    # ВАШ КОД ЗДЕСЬ
    # 1. Конвертируйте RGB → HSV
    # 2. Разделите каналы H, S, V
    # 3. Умножьте S на factor, ограничьте диапазон [0, 255]
    # 4. Соберите каналы обратно
    # 5. Конвертируйте HSV → RGB
    pass


# Тестирование с тремя значениями factor
saturation_factors = [0.3, 1.0, 1.7]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(img_rgb)
axes[0].set_title('Оригинал', fontsize=14)
axes[0].axis('off')

for i, factor in enumerate(saturation_factors, start=1):
    result = adjust_saturation(img_rgb, factor)
    axes[i].imshow(result)
    if factor < 1:
        axes[i].set_title(f'Saturation × {factor}\n(Обесцвечивание)', fontsize=14)
    elif factor > 1:
        axes[i].set_title(f'Saturation × {factor}\n(Усиление)', fontsize=14)
    else:
        axes[i].set_title(f'Saturation × {factor}\n(Без изменений)', fontsize=14)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

---

### Задание 1.5: Фильтр "Сепия"

**Теория:**

Эффект **сепия** имитирует старые фотографии с теплым коричневатым оттенком.

Реализуется через **матричное преобразование** RGB:

$$
\begin{bmatrix}
R_{\text{sepia}} \\
G_{\text{sepia}} \\
B_{\text{sepia}}
\end{bmatrix}
=
\begin{bmatrix}
0.393 & 0.769 & 0.189 \\
0.349 & 0.686 & 0.168 \\
0.272 & 0.534 & 0.131
\end{bmatrix}
\cdot
\begin{bmatrix}
R \\
G \\
B
\end{bmatrix}
$$

**В коде:**

```python
R_sepia = 0.393*R + 0.769*G + 0.189*B
G_sepia = 0.349*R + 0.686*G + 0.168*B
B_sepia = 0.272*R + 0.534*G + 0.131*B
```

**Особенности:**
- Результат может превысить 255 → нужно `np.clip()`
- Можно регулировать **интенсивность эффекта** через смешивание с оригиналом:

$$
I_{\text{result}} = \text{intensity} \cdot I_{\text{sepia}} + (1 - \text{intensity}) \cdot I_{\text{original}}
$$

**Задача:**

Реализуйте функцию `apply_sepia(image, intensity)` с регулируемой интенсивностью эффекта. 

А сможете ли это сделать как матричное умножение на чистом numpy?

---

In [ ]:
def apply_sepia(image: np.ndarray, intensity: float = 1.0) -> np.ndarray:
    """
    Применение фильтра сепия.
    
    Args:
        image: Входное изображение (RGB)
        intensity: Интенсивность эффекта (0.0 - 1.0)
    
    Returns:
        Изображение с эффектом сепия
    """
    # ВАШ КОД ЗДЕСЬ
    # 1. Разделите каналы R, G, B
    # 2. Вычислите новые каналы по формулам выше
    # 3. Объедините в одно изображение с помощью np.stack
    # 4. Ограничьте диапазон [0, 255]
    # 5. Если intensity < 1.0, смешайте с оригиналом:
    #    result = intensity * sepia + (1 - intensity) * original
    # 6. Верните как uint8
    pass


# Тестирование с тремя уровнями интенсивности
intensity_values = [0.3, 0.7, 1.0]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(img_rgb)
axes[0].set_title('Оригинал', fontsize=14)
axes[0].axis('off')

for i, intensity in enumerate(intensity_values, start=1):
    result = apply_sepia(img_rgb, intensity)
    axes[i].imshow(result)
    axes[i].set_title(f'Sepia (intensity={intensity})', fontsize=14)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

---

### Задание 1.6: Работа с кривыми цвета (Gamma Correction)

**Теория:**

**Гамма-коррекция** — это нелинейное преобразование яркости, которое используется для:
- Коррекции отображения на разных устройствах
- Улучшения визуального восприятия
- Имитации эффектов пере/недоэкспонирования

**Формула:**

$$
I_{\text{out}} = 255 \cdot \left( \frac{I_{\text{in}}}{255} \right)^{\gamma}
$$

или эквивалентно:

$$
I_{\text{out}} = 255 \cdot \left( \frac{I_{\text{in}}}{255} \right)^{1/\gamma}
$$

**Эффекты:**
- **γ < 1** (например, 0.5) → изображение **светлеет**, темные области становятся ярче
- **γ = 1** → изображение не изменяется (линейная зависимость)
- **γ > 1** (например, 2.0) → изображение **темнеет**, светлые области становятся темнее

**Визуализация кривой:**

```
Output ↑
255    |     γ < 1 (светлее)
       |    /
       |   /  γ = 1 (без изменений)
       |  /  /
       | /  /
       |/  /   γ > 1 (темнее)
       |  /   /
       | /   /
   0   |/_________→ Input (0-255)
       0          255
```

**Look-Up Table (LUT):**

Для ускорения можно предвычислить таблицу преобразования:

```python
lut = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype(np.uint8)
result = cv2.LUT(image, lut)
```

**Задача:**

Реализуйте функцию `apply_gamma_correction(image, gamma)` с использованием LUT.

---

In [ ]:
def apply_gamma_correction_lut(image: np.ndarray, gamma: float) -> np.ndarray:
    """
    Применение гамма-коррекции к изображению.
    
    Args:
        image: Входное изображение (RGB)
        gamma: Параметр гамма (< 1 светлеет, > 1 темнеет)
    
    Returns:
        Изображение с гамма-коррекцией
    """
    # ВАШ КОД ЗДЕСЬ
    # 1. Создайте Look-Up Table (LUT):
    # 2. Примените LUT к изображению:
    # 3. Верните результат
    pass

def apply_gamma_correction(image: np.ndarray, gamma: float) -> np.ndarray:
    """
    Применение гамма-коррекции к изображению.
    
    Args:
        image: Входное изображение (RGB)
        gamma: Параметр гамма (< 1 светлеет, > 1 темнеет)
    
    Returns:
        Изображение с гамма-коррекцией
    """
    # ВАШ КОД ЗДЕСЬ
    # Сделайте честное преобразование
    pass

# ВАШ КОД ЗДЕСЬ
# Сравните скорость работы и результаты двух функций выше.


# Тестирование с тремя значениями gamma
gamma_values = [0.5, 1.0, 2.2]

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Первый ряд: результаты
axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Оригинал', fontsize=14)
axes[0, 0].axis('off')

for i, gamma in enumerate(gamma_values, start=1):
    result = apply_gamma_correction(img_rgb, gamma)
    axes[0, i].imshow(result)
    if gamma < 1:
        axes[0, i].set_title(f'γ = {gamma}\n(Светлее)', fontsize=14)
    elif gamma > 1:
        axes[0, i].set_title(f'γ = {gamma}\n(Темнее)', fontsize=14)
    else:
        axes[0, i].set_title(f'γ = {gamma}\n(Без изменений)', fontsize=14)
    axes[0, i].axis('off')

# Второй ряд: кривые гамма-коррекции
x = np.linspace(0, 255, 256)
for i in range(4):
    if i == 0:
        # Пустая ячейка для выравнивания
        axes[1, i].axis('off')
    else:
        gamma = gamma_values[i-1]
        y = 255 * ((x / 255.0) ** gamma)
        axes[1, i].plot(x, y, 'r-', linewidth=2, label=f'γ={gamma}')
        axes[1, i].plot(x, x, 'b--', linewidth=1, alpha=0.5, label='γ=1 (linear)')
        axes[1, i].set_xlabel('Input', fontsize=12)
        axes[1, i].set_ylabel('Output', fontsize=12)
        axes[1, i].set_title(f'Кривая γ={gamma}', fontsize=12)
        axes[1, i].legend()
        axes[1, i].grid(alpha=0.3)
        axes[1, i].set_xlim(0, 255)
        axes[1, i].set_ylim(0, 255)

plt.tight_layout()
plt.show()

---

### 🎯 Бонусное задание: Комбинированная обработка

**Задача:**

Создайте свой собственный **фото-фильтр**, комбинируя изученные преобразования.

**Примеры идей:**

1. **"Винтажное фото":**
   - Уменьшить насыщенность (factor = 0.7)
   - Применить легкую сепию (intensity = 0.5)
   - Снизить контраст (alpha = 0.9)
   - Добавить немного яркости (beta = 10)

2. **"Драматическое фото":**
   - Увеличить контраст (alpha = 1.3)
   - Затемнить (beta = -20)
   - Увеличить насыщенность (factor = 1.2)
   - Гамма-коррекция (gamma = 1.2)

3. **"Солнечное фото":**
   - Осветлить (gamma = 0.7)
   - Увеличить насыщенность (factor = 1.3)
   - Добавить яркости (beta = 20)

**Задание:**

Создайте функцию `custom_filter(image)`, которая применяет вашу уникальную комбинацию эффектов. Используйте ваш фильтр для редактирования  фотографии города `test_image_4_1.jpg`

---

In [ ]:
def custom_filter(image: np.ndarray) -> np.ndarray:
    """
    Ваш собственный фото-фильтр.
    
    Комбинируйте изученные функции для создания уникального эффекта!
    """
    # ВАШ КОД ЗДЕСЬ
    # Примените несколько преобразований последовательно
    # Экспериментируйте с параметрами!
    
    return result


# Визуализация вашего фильтра
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].imshow(img_rgb)
axes[0].set_title('Оригинал', fontsize=16)
axes[0].axis('off')

result = custom_filter(img_rgb)
axes[1].imshow(result)
axes[1].set_title('Ваш Кастомный Фильтр', fontsize=16)
axes[1].axis('off')

plt.tight_layout()
plt.show()

---

## 📚 Итоги Части 1

### Что вы реализовали:

1. ✅ **Регулировка яркости** — линейный сдвиг значений пикселей
2. ✅ **Регулировка контрастности** — масштабирование значений пикселей
3. ✅ **Конвертация в grayscale** — три метода (average, luminosity, lightness)
4. ✅ **Регулировка насыщенности** — работа с цветовым пространством HSV
5. ✅ **Фильтр сепия** — матричное преобразование RGB каналов
6. ✅ **Гамма-коррекция** — нелинейное преобразование с использованием LUT
7. ✅ **Кастомный фильтр** — комбинация всех изученных техник

---

### 💡 Ключевые концепции:

**Линейные преобразования:**
- Яркость: `I_new = I + β`
- Контрастность: `I_new = α × I`
- Комбо: `I_new = α × I + β`

**Цветовые пространства:**
- **RGB** — Red, Green, Blue (аддитивная модель)
- **HSV** — Hue, Saturation, Value (интуитивная для человека)
- **Grayscale** — одноканальное представление яркости

**Нелинейные преобразования:**
- Гамма-коррекция: `I_new = 255 × (I/255)^γ`
- Look-Up Tables (LUT) для ускорения

**Важные приемы:**
- `np.clip()` для ограничения диапазона [0, 255]
- Преобразование типов: `uint8` ↔ `int16` ↔ `float32`
- `cv2.LUT()` для быстрого применения таблицы преобразований

---

---

## Часть 2: Перспективные преобразования и работа с 3D объектами

**Цель:** Научиться работать с перспективными преобразованиями для наложения 2D изображений на 3D поверхности.

**Задачи:**
1. Создать синтетическое изображение кубика с известными координатами углов
2. Сгенерировать изображения с цифрами 1, 2, 3
3. Применить перспективное преобразование для "натягивания" цифр на грани кубика

**Теория:**

### Перспективное преобразование (Perspective Transform)

Перспективное преобразование - это преобразование, которое сохраняет прямые линии, но не сохраняет параллельность. Оно описывается матрицей $3 \times 3$:

$$
\begin{bmatrix}
x' \\
y' \\
w'
\end{bmatrix}
=
\begin{bmatrix}
h_{11} & h_{12} & h_{13} \\
h_{21} & h_{22} & h_{23} \\
h_{31} & h_{32} & h_{33}
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
$$

Итоговые координаты: $x_{new} = \frac{x'}{w'}$, $y_{new} = \frac{y'}{w'}$

**В OpenCV:**
- `cv2.getPerspectiveTransform(src, dst)` — вычисляет матрицу преобразования по 4 парам точек
- `cv2.warpPerspective(img, M, size)` — применяет преобразование к изображению

### Почему 4 точки?

Матрица гомографии имеет 8 степеней свободы (9 элементов - 1 нормализация). Каждая пара точек дает 2 уравнения:

$$
x_i' = \frac{h_{11}x_i + h_{12}y_i + h_{13}}{h_{31}x_i + h_{32}y_i + h_{33}}
$$

$$
y_i' = \frac{h_{21}x_i + h_{22}y_i + h_{23}}{h_{31}x_i + h_{32}y_i + h_{33}}
$$

Для 8 неизвестных нужно 4 пары точек → 8 уравнений.

### Задание 2.1: Создание синтетического изображения кубика

Создайте функцию, которая генерирует изображение кубика в почти-изометрической проекции.

**Требования:**
- Разные цвета граней для наглядности
- Возврат: изображение + словарь с координатами углов каждой грани

In [ ]:
##### from typing import List, Dict

# Функция для преобразования 3D в 2D (почти-изометрическая проекция)
def project_3d_to_2d(point_3d):
    """Преобразует 3D точку в 2D используя изометрическую проекцию."""
    x, y, z = point_3d
    # Изометрическая проекция
    x_2d = y - x * 0.3 * np.sin(np.radians(30))
    y_2d = x * 0.3 * np.cos(np.radians(30)) - z
    return np.array([x_2d, y_2d])
        
def create_cube(img_width: int = 800, 
                img_height: int = 600) -> Tuple[np.ndarray, Dict[str, List]]:
    """
    Создает изображение кубика в изометрической проекции.

    Изометрическая проекция преобразует 3D координаты (x, y, z) в 2D:
        x_2d = (x - z) * cos(30°)
        y_2d = y + (x + z) * sin(30°)

    где cos(30°) ≈ 0.866, sin(30°) = 0.5

    Args:
        cube_size: размер стороны кубика в пикселях
        img_width: ширина итогового изображения
        img_height: высота итогового изображения

    Returns:
        image: RGB изображение кубика
        corners: словарь с координатами углов граней
                {'top': [...], 'right': [...], 'left': [...]}
    """
    # Создаем белое изображение
    image = np.ones((img_height, img_width, 3), dtype=np.uint8) * 255

    # Определяем 8 вершин куба в 3D координатах
    s = 200
    vertices_3d = np.array([
        [0, 0, 0],  # 0: нижний левый задний
        [s, 0, 0],  # 1: нижний правый задний
        [s, s, 0],  # 2: верхний правый задний
        [0, s, 0],  # 3: верхний левый задний
        [0, 0, s],  # 4: нижний левый передний
        [s, 0, s],  # 5: нижний правый передний
        [s, s, s],  # 6: верхний правый передний
        [0, s, s],  # 7: верхний левый передний
    ], dtype=np.float32) # - s//2

    # Преобразуем все вершины в 2D
    vertices_2d = np.array([project_3d_to_2d(v) for v in vertices_3d])

    # Центрируем и смещаем в центр изображения
    center_x = img_width // 2
    center_y = img_height // 2

    vertices_2d[:, 0] += center_x
    vertices_2d[:, 1] += center_y # - vertices_2d[:, 1]  # Инвертируем Y (OpenCV)

    # Преобразуем в целые числа для рисования
    vertices_2d = vertices_2d.astype(np.int32)

    # Определяем три видимые грани кубика
    # Важно: порядок точек определяет заполнение (по часовой стрелке)

    # Передняя грань (front) - вершины 1, 2, 6, 5
    front_face = vertices_2d[[1, 2, 6, 5]]

    # Правая грань (right) - вершины 2, 3, 7, 6
    right_face = vertices_2d[[2, 3, 7, 6]]

    # Верхняя грань (top) - вершины 4, 5, 7, 6 (x = 0)
    top_face = vertices_2d[[4, 5, 6, 7]]

    # Рисуем грани разными цветами
    # Верхняя грань - светло-голубая
    cv2.fillPoly(image, [top_face], color=(255, 220, 180))

    # Правая грань - светло-розовая
    cv2.fillPoly(image, [right_face], color=(220, 180, 255))

    # Передняя грань - светло-зелёная
    cv2.fillPoly(image, [front_face], color=(180, 255, 220))

        
    for center in vertices_2d:
        image = cv2.circle(image, center=center, radius=3, color=(255, 0, 0), thickness=3)
    

    # Рисуем рёбра кубика чёрным цветом
    edges = [
        # Нижние рёбра (z=0)
        (0, 1), (1, 2), (2, 3), (3, 0),
        # Верхние рёбра (z=s)
        (4, 5), (5, 6), (6, 7), (7, 4),
        # Вертикальные рёбра
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]

    for start, end in edges:
        pt1 = tuple(vertices_2d[start])
        pt2 = tuple(vertices_2d[end])

        if (vertices_3d[start] == 0).all() or (vertices_3d[end] == 0).all():
            color = (128, 128, 128)
            thickness=1
        else:
            color = (0, 0, 0)
            thickness=2
        
        cv2.line(image, pt1, pt2, color=color, thickness=thickness)

    # Формируем словарь с координатами углов граней
    corners = {
        'top': top_face.tolist(),
        'right': right_face.tolist(),
        'front': front_face.tolist()
    }

    return image, corners

# Создаем кубик
cube_img, cube_corners = create_cube()

# Визуализация
plt.figure(figsize=(10, 8))
plt.imshow(cube_img)
plt.title('Синтетический кубик с известными координатами')
plt.axis('off')

# Выводим координаты углов
print("\nКоординаты углов граней:")
for face_name, corners in cube_corners.items():
    print(f"\n{face_name.upper()}:")
    for i, corner in enumerate(corners):
        print(f"  Угол {i+1}: ({corner[0]:.1f}, {corner[1]:.1f})")

plt.tight_layout()
plt.show()

### Задание 2.2: Генерация изображений с цифрами

Создайте функцию для генерации изображений с цифрами 1, 2, 3 разными цветами.

**Работа с текстом в OpenCV:**

```python
cv2.putText(img, text, org, fontFace, fontScale, color, thickness)
```

**Для использования TrueType шрифтов:**

```python
from PIL import Image, ImageDraw, ImageFont

# Создание изображения
pil_img = Image.new('RGB', (width, height), color='white')
draw = ImageDraw.Draw(pil_img)

# Загрузка шрифта
font = ImageFont.truetype('path/to/font.ttf', size)

# Рисование текста
draw.text((x, y), text, font=font, fill=color)

# Конвертация в NumPy
np_img = np.array(pil_img)
```

**Требования:**
- Размер изображения: 400×400 пикселей
- Белый фон
- Цифры центрированы
- Цвета: 1 - красный (255,0,0), 2 - зеленый (0,255,0), 3 - синий (0,0,255)
- Используйте шрифт `./DejaVuSans.ttf`

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def create_digit_image(digit: int, 
                      color: Tuple[int, int, int],
                      size: int = 400,
                      font_path: str = './DejaVuSans.ttf') -> np.ndarray:
    """
    Создает изображение с цифрой.
    
    Args:
        digit: цифра для отображения (1, 2, или 3)
        color: цвет цифры в формате RGB (tuple)
        size: размер изображения (квадратное)
        font_path: путь к файлу шрифта TrueType
    
    Returns:
        image: RGB изображение с цифрой
    """
    # ВАШ КОД ЗДЕСЬ
    # Подсказки:
    # 1. Создайте PIL изображение: Image.new('RGB', (size, size), 'white')
    # 2. Создайте объект для рисования: ImageDraw.Draw(img)
    # 3. Загрузите шрифт: ImageFont.truetype(font_path, font_size)
    #    Подберите font_size так, чтобы цифра занимала ~70% изображения
    # 4. Для центрирования используйте:
    #    bbox = draw.textbbox((0, 0), str(digit), font=font)
    #    text_width = bbox[2] - bbox[0]
    #    text_height = bbox[3] - bbox[1]
    #    x = (size - text_width) // 2
    #    y = (size - text_height) // 2
    # 5. Нарисуйте текст: draw.text((x, y), str(digit), font=font, fill=color)
    # 6. Конвертируйте в NumPy: np.array(pil_img)
    # 7. Конвертируйте из RGB в BGR для OpenCV: cv2.cvtColor(..., cv2.COLOR_RGB2BGR)
    
    pass

# Генерируем изображения с цифрами
colors = {
    1: (255, 0, 0),      # Красный
    2: (0, 255, 0),      # Зеленый
    3: (0, 0, 255)       # Синий
}

digit_images = {}
for digit, color in colors.items():
    digit_images[digit] = create_digit_image(digit, color)

# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for idx, (digit, img) in enumerate(digit_images.items()):
    axes[idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(f'Цифра {digit}', fontsize=14)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\nГенерация цифр завершена!")
print(f"Размеры изображений: {digit_images[1].shape}")

### Задание 2.3: Наложение цифр на грани кубика

Используйте перспективное преобразование для "натягивания" цифр на грани кубика.

**Алгоритм:**

1. **Определить исходные точки** (src_points) - углы изображения цифры:
   ```
   (0, 0)           (w, 0)
      ┌──────────────┐
      │              │
      │              │
      └──────────────┘
   (0, h)           (w, h)
   ```

2. **Определить целевые точки** (dst_points) - углы грани кубика (из `cube_corners`)

3. **Вычислить матрицу перспективного преобразования:**
   ```python
   M = cv2.getPerspectiveTransform(src_points, dst_points)
   ```

4. **Применить преобразование:**
   ```python
   warped = cv2.warpPerspective(digit_img, M, (width, height))
   ```

5. **Создать маску** для наложения только цифры (без белого фона)

6. **Наложить преобразованное изображение** на кубик

**Соответствие:**
- Цифра 1 → Верхняя грань (top)
- Цифра 2 → Правая грань (right)
- Цифра 3 → Левая грань (left)

In [ ]:
def warp_digit_to_face(digit_img: np.ndarray,
                      face_corners: np.ndarray,
                      output_size: Tuple[int, int]) -> np.ndarray:
    """
    Применяет перспективное преобразование для наложения цифры на грань.
    
    Args:
        digit_img: изображение с цифрой
        face_corners: 4 угла грани кубика в порядке: top-left, top-right, 
                     bottom-right, bottom-left
        output_size: размер выходного изображения (width, height)
    
    Returns:
        warped: преобразованное изображение
    """
    # ВАШ КОД ЗДЕСЬ
    # Подсказки:
    # 1. Получите размеры изображения цифры: h, w = digit_img.shape[:2]
    # 2. Определите исходные точки (4 угла изображения цифры):
    #    src_points = np.float32(???)
    # 3. Целевые точки - это углы грани:
    #    dst_points = np.float32(face_corners)
    # 4. Вычислите матрицу преобразования:
    #    M = cv2.getPerspectiveTransform(???)
    # 5. Примените преобразование:
    #    warped = cv2.warpPerspective(???)
    # 6. Верните результат
    
    pass

def overlay_image(background: np.ndarray,
                 overlay: np.ndarray,
                 threshold: int = 250) -> np.ndarray:
    """
    Накладывает изображение overlay на background, игнорируя белые пиксели.
    
    Args:
        background: базовое изображение
        overlay: накладываемое изображение
        threshold: порог для определения "белого" цвета
    
    Returns:
        result: результат наложения
    """
    # ВАШ КОД ЗДЕСЬ
    # Подсказки:
    # 1. Скопируйте background: result = background.copy()
    # 2. Создайте маску для не-белых пикселей:
    #    mask = np.all(overlay < threshold, axis=2)
    # 3. Скопируйте пиксели overlay в result там, где mask == True:
    #    result[mask] = overlay[mask]
    # 4. Верните результат
    
    pass

# Применяем перспективное преобразование для каждой грани
output_size = (cube_img.shape[1], cube_img.shape[0])  # (width, height)

# Создаем копию изображения кубика
result_img = cube_img.copy()

# Соответствие: грань -> цифра
face_digit_mapping = {
    'top': 1,
    'right': 2,
    'left': 3
}

# Накладываем цифры на грани
for face_name, digit in face_digit_mapping.items():
    print(f"Накладываем цифру {digit} на грань '{face_name}'...")
    
    # Получаем углы грани
    corners = cube_corners[face_name]
    
    # Применяем перспективное преобразование
    warped_digit = warp_digit_to_face(digit_images[digit], corners, output_size)
    
    # Накладываем на результат
    result_img = overlay_image(result_img, warped_digit)

print("\nГотово!")

# Визуализация результата
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(cube_img)
axes[0].set_title('Исходный кубик', fontsize=14)
axes[0].axis('off')

axes[1].imshow(result_img)
axes[1].set_title('Кубик с цифрами на гранях', fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

### Визуализация промежуточных результатов

Давайте посмотрим на каждую грань отдельно, чтобы лучше понять процесс преобразования.

In [ ]:
# Визуализируем каждую грань отдельно
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, (face_name, digit) in enumerate(face_digit_mapping.items()):
    # Исходная цифра
    axes[0, idx].imshow(cv2.cvtColor(digit_images[digit], cv2.COLOR_BGR2RGB))
    axes[0, idx].set_title(f'Исходная цифра {digit}', fontsize=12)
    axes[0, idx].axis('off')
    
    # Преобразованная цифра
    corners = cube_corners[face_name]
    warped = warp_digit_to_face(digit_images[digit], corners, output_size)
    
    axes[1, idx].imshow(warped)
    axes[1, idx].set_title(f'После преобразования (грань {face_name})', fontsize=12)
    axes[1, idx].axis('off')
    
    # Рисуем углы грани для наглядности
    for corner in corners:
        axes[1, idx].plot(corner[0], corner[1], 'ro', markersize=8)

plt.tight_layout()
plt.show()

---

## 📊 Итоги Части 2

### Что мы изучили:

✅ **Перспективные преобразования**
- Математика гомографии (матрица $3 \times 3$)
- Почему нужно 4 пары точек
- `cv2.getPerspectiveTransform()` и `cv2.warpPerspective()`

✅ **Работа с синтетическими данными**
- Генерация 3D объектов (изометрическая проекция)
- Работа с TrueType шрифтами через PIL
- Интеграция PIL и OpenCV

✅ **Композитинг изображений**
- Создание масок по цветовому порогу
- Наложение изображений с прозрачностью
- Работа с альфа-каналом


# Компьютерное зрение: Семинар
## Часть 3: Аугментации изображений с Albumentations

**Преподаватель:** Иванов Александр, Т-Банк

**Аудитория:** ФКН ВШЭ, МФТИ

**Цель:** Изучить популярные аугментации из библиотеки Albumentations и реализовать их с нуля, используя OpenCV и NumPy.

**Что вы узнаете:**
- Как работают различные аугментации изнутри
- Математику и алгоритмы за каждой трансформацией
- Практическую реализацию с использованием OpenCV
- Как применять аугментации для улучшения моделей компьютерного зрения

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from typing import Tuple, Optional

# Настройка отображения
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

print("Библиотеки загружены успешно!")
print(f"OpenCV версия: {cv2.__version__}")
print(f"NumPy версия: {np.__version__}")
print(f"Albumentations версия: {A.__version__}")

In [ ]:
# Вспомогательная функция для resize
def resize_to_max_side(image: np.ndarray, max_side: int = 512) -> np.ndarray:
    """Изменяет размер изображения так, чтобы большая сторона была равна max_side."""
    h, w = image.shape[:2]
    if h > w:
        new_h = max_side
        new_w = int(w * max_side / h)
    else:
        new_w = max_side
        new_h = int(h * max_side / w)
    return cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

# Загружаем тестовые изображения
# Для геометрических преобразований
img_geom_bgr = cv2.imread('test_image_2_1.jpg')
img_geom_rgb = cv2.cvtColor(img_geom_bgr, cv2.COLOR_BGR2RGB)
img_geom_rgb = resize_to_max_side(img_geom_rgb, 512)

# Для цветовых преобразований
img_color_bgr = cv2.imread('test_image_1_1.jpg')
img_color_rgb = cv2.cvtColor(img_color_bgr, cv2.COLOR_BGR2RGB)
img_color_rgb = resize_to_max_side(img_color_rgb, 512)

print(f"Геометрическое изображение: {img_geom_rgb.shape}")
print(f"Цветовое изображение: {img_color_rgb.shape}")

# Отображаем оригиналы
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(img_geom_rgb)
axes[0].set_title('Тестовое изображение для геометрических аугментаций', fontsize=12)
axes[0].axis('off')

axes[1].imshow(img_color_rgb)
axes[1].set_title('Тестовое изображение для цветовых аугментаций', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Вспомогательная функция для визуализации сравнений
def visualize_comparison(image: np.ndarray, 
                         custom_func, 
                         albumentations_transform,
                         title: str,
                         params: dict = None):
    """
    Создает коллаж 2x3: 
    - Верхний ряд: 3 применения пользовательской функции
    - Нижний ряд: 3 применения albumentations трансформации
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Верхний ряд - наша реализация
    for i in range(3):
        if params is not None:
            result = custom_func(image.copy(), **params)
        else:
            result = custom_func(image.copy())
        axes[0, i].imshow(result)
        axes[0, i].set_title(f'Наша реализация #{i+1}', fontsize=11)
        # axes[0, i].axis('off')
    
    # Нижний ряд - albumentations
    for i in range(3):
        result = albumentations_transform(image=image.copy())['image']
        axes[1, i].imshow(result)
        axes[1, i].set_title(f'Albumentations #{i+1}', fontsize=11)
        # axes[1, i].axis('off')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---

## 1. RGBShift — Сдвиг RGB каналов

### Описание и математика

**RGBShift** — это простая аугментация, которая независимо сдвигает значения каждого из трех цветовых каналов (Red, Green, Blue) на случайную величину.

**Математическая формула:**

Для каждого канала $c \in \{R, G, B\}$:

$$
I_c^{\text{new}}(x, y) = \text{clip}\left(I_c^{\text{old}}(x, y) + \Delta_c, 0, 255\right)
$$

где:
- $I_c^{\text{old}}(x, y)$ — исходное значение пикселя в канале $c$
- $\Delta_c$ — случайный сдвиг для канала $c$, обычно $\Delta_c \in [-\text{limit}, +\text{limit}]$
- $\text{clip}(v, 0, 255)$ — ограничение значения диапазоном $[0, 255]$

**Параметры:**
- `r_shift_limit`: диапазон сдвига для красного канала (например, [-20, 20])
- `g_shift_limit`: диапазон сдвига для зеленого канала
- `b_shift_limit`: диапазон сдвига для синего канала

**Применение:**
- Симуляция различных условий освещения
- Имитация цветового дисбаланса камеры
- Повышение робастности модели к вариациям цвета

### Полезные функции OpenCV

- `np.random.randint(low, high)` или `np.random.uniform(low, high)` — генерация случайных сдвигов
- `np.clip(array, min, max)` — ограничение значений диапазоном
- Прямая работа с каналами: `image[:, :, 0]`, `image[:, :, 1]`, `image[:, :, 2]`

---

In [ ]:
def custom_rgb_shift(image: np.ndarray, 
                     r_shift_limit: int = 20,
                     g_shift_limit: int = 20,
                     b_shift_limit: int = 20) -> np.ndarray:
    """
    Применяет случайный сдвиг к каждому из RGB каналов.
    
    Args:
        image: Входное RGB изображение
        r_shift_limit: Максимальный сдвиг для красного канала
        g_shift_limit: Максимальный сдвиг для зеленого канала
        b_shift_limit: Максимальный сдвиг для синего канала
    
    Returns:
        Изображение с примененным RGBShift
    """
    # ВАШ КОД ЗДЕСЬ
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_rgb_shift,
    albumentations_transform=A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1.0),
    title="RGBShift: Сравнение реализаций",
    params={'r_shift_limit': 20, 'g_shift_limit': 20, 'b_shift_limit': 20}
)

---

## 2. Downscale — Понижение разрешения

### Описание и математика

**Downscale** — аугментация, которая уменьшает разрешение изображения, а затем масштабирует его обратно к исходному размеру. Это имитирует изображения низкого качества и помогает модели быть устойчивой к различным разрешениям.

**Математический процесс:**

1. **Downsampling (понижение):**
   $$I_{\text{low}} = \text{resize}(I_{\text{original}}, (W \cdot s, H \cdot s))$$
   где $s \in [s_{\min}, s_{\max}]$ — коэффициент масштабирования (например, $s=0.5$ уменьшает в 2 раза)

2. **Upsampling (восстановление):**
   $$I_{\text{result}} = \text{resize}(I_{\text{low}}, (W, H))$$

**Методы интерполяции:**
- `cv2.INTER_NEAREST` — ближайший сосед (самый быстрый, создает пиксельный эффект)
- `cv2.INTER_LINEAR` — билинейная интерполяция (хороший баланс скорость/качество)
- `cv2.INTER_AREA` — интерполяция по области (лучше для уменьшения)
- `cv2.INTER_CUBIC` — бикубическая интерполяция (медленнее, но качественнее)

**Физический смысл:**
- При уменьшении теряется информация о высоких частотах (детали)
- При увеличении обратно эти детали не восстанавливаются
- Результат — размытое изображение с потерей четкости

**Применение:**
- Имитация камер с низким разрешением
- Устойчивость к JPEG-сжатию
- Обучение на данных разного качества
- Эмуляция передачи изображений по медленным каналам



In [ ]:
def custom_downscale(image: np.ndarray,
                     scale_min: float = 0.25,
                     scale_max: float = 0.5,
                     interpolation_down=cv2.INTER_NEAREST,
                     interpolation_up=cv2.INTER_LINEAR) -> np.ndarray:
    """
    Понижает разрешение изображения и восстанавливает обратно.
    
    Args:
        image: Входное изображение
        scale_min: Минимальный коэффициент масштабирования
        scale_max: Максимальный коэффициент масштабирования
        interpolation_down: Метод интерполяции при уменьшении
        interpolation_up: Метод интерполяции при увеличении
    
    Returns:
        Изображение с пониженным и восстановленным разрешением
    """
    # TODO: Реализуйте логику downscale
    # 1. Получите размеры изображения (height, width)
    # 2. Выберите случайный коэффициент масштабирования между scale_min и scale_max
    # 3. Вычислите новые размеры (не забудьте преобразовать к int)
    # 4. Уменьшите изображение используя cv2.resize с interpolation_down
    # 5. Увеличьте обратно до исходного размера используя interpolation_up
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_downscale,
    albumentations_transform=A.Downscale(scale_min=0.25, scale_max=0.5, p=1.0),
    title="Downscale: Сравнение реализаций",
    params={'scale_min': 0.25, 'scale_max': 0.5}
)


---

## 3. Equalize — Эквализация гистограммы

### Описание и математика

**Histogram Equalization (Эквализация гистограммы)** — это метод улучшения контраста изображения путем перераспределения интенсивностей пикселей так, чтобы гистограмма стала более равномерной.

**Алгоритм:**

1. **Вычисление гистограммы**: Подсчитываем количество пикселей для каждого уровня яркости $k \in [0, 255]$:
   $$h(k) = \text{count}(\text{pixels with intensity } k)$$

2. **Нормализованная гистограмма** (вероятность):
   $$p(k) = \frac{h(k)}{N}$$
   где $N$ — общее количество пикселей.

3. **Кумулятивная функция распределения (CDF)**:
   $$\text{CDF}(k) = \sum_{i=0}^{k} p(i)$$

4. **Преобразование интенсивности**:
   $$I_{\text{new}}(x, y) = \text{round}\left(255 \cdot \text{CDF}(I_{\text{old}}(x, y))\right)$$

**Для цветных изображений:**
- Конвертируем в HSV или YCrCb
- Применяем эквализацию только к каналу яркости (V или Y)
- Конвертируем обратно в RGB

**Применение:**
- Улучшение контраста на темных/светлых изображениях
- Нормализация освещения
- Подготовка изображений для анализа

### Полезные функции OpenCV

- `cv2.equalizeHist(channel)` — эквализация для одноканального изображения
- `cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)` — конвертация в YCrCb
- `cv2.cvtColor(image, cv2.COLOR_YCrCb2RGB)` — обратная конвертация

---

In [ ]:
def custom_equalize(image: np.ndarray, mode: str = 'cv') -> np.ndarray:
    """
    Применяет эквализацию гистограммы к изображению.
    
    Args:
        image: Входное RGB изображение
        mode: 'cv' для использования cv2.equalizeHist, 
              'pil' для эквализации всех каналов по отдельности
    
    Returns:
        Изображение после эквализации
    """
    if mode == 'cv':
        # Конвертируем в YCrCb (аналог albumentations)
        # Применяем эквализацию только к Y каналу (яркость)
        # Конвертируем обратно в RGB
        
    elif mode == 'pil':
        # Эквализация каждого канала отдельно
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_equalize,
    albumentations_transform=A.Equalize(mode='cv', p=1.0),
    title="Equalize: Сравнение реализаций",
    params={'mode': 'cv'}
)

---

## 4. Posterize — Постеризация

### Описание и математика

**Posterize** — аугментация, которая уменьшает количество бит на канал, создавая эффект постера с ограниченным числом цветов.

**Математическая формула:**

Для каждого пикселя и каждого канала применяется квантование:

$$I'(x,y,c) = \left\lfloor \frac{I(x,y,c)}{2^{8-n}} \right\rfloor \times 2^{8-n}$$

где:
- $I(x,y,c)$ — значение пикселя в канале $c$ (0-255)
- $n$ — количество бит на канал (обычно 1-8)
- $2^{8-n}$ — размер шага квантования
- $\lfloor \cdot \rfloor$ — округление вниз

**Количество уровней:**
- При $n$ битах возможно $2^n$ уникальных значений на канал
- Всего цветов в RGB: $(2^n)^3 = 2^{3n}$

**Примеры:**
- $n=8$ (без изменений): 256 уровней, $2^{24} \approx 16.7$ млн цветов
- $n=4$: 16 уровней, $2^{12} = 4096$ цветов
- $n=2$: 4 уровня, $2^6 = 64$ цвета
- $n=1$: 2 уровня, $2^3 = 8$ цветов

**Пример расчета:**
Для $n=3$ (8 уровней):
- Шаг: $2^{8-3} = 2^5 = 32$
- Значение 127: $\lfloor 127/32 \rfloor \times 32 = 3 \times 32 = 96$
- Значение 200: $\lfloor 200/32 \rfloor \times 32 = 6 \times 32 = 192$

**Применение:**
- Имитация низкокачественных дисплеев
- Устойчивость к квантованию цвета
- Художественная стилизация
- Эмуляция старых графических систем



In [ ]:
def custom_posterize(image: np.ndarray,
                     num_bits: int = 4) -> np.ndarray:
    """
    Применяет постеризацию — уменьшение количества бит на канал.
    
    Args:
        image: Входное изображение (H, W, C)
        num_bits: Количество бит на канал (1-8)
    
    Returns:
        Постеризованное изображение
    """
    # TODO: Реализуйте постеризацию
    # 1. Вычислите размер шага квантования
    # 2. Квантуйте изображение: для каждого пикселя:
    # 3. Убедитесь, что результат в диапазоне [0, 255]
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_posterize,
    albumentations_transform=A.Posterize(num_bits=4, p=1.0),
    title="Posterize: Сравнение реализаций",
    params={'num_bits': 4}
)


---

## 5. CLAHE — Adaptive Histogram Equalization

### Описание и математика

**CLAHE (Contrast Limited Adaptive Histogram Equalization)** — это улучшенная версия эквализации гистограммы, которая:
1. Разбивает изображение на небольшие блоки (tiles)
2. Применяет эквализацию к каждому блоку отдельно
3. Использует билинейную интерполяцию для сглаживания границ блоков
4. Ограничивает усиление контраста параметром `clip_limit`

**Алгоритм:**

1. **Разбиение на блоки**: Изображение делится на сетку $M \times N$ блоков (tiles).

2. **Ограничение контраста**: Для каждого блока строится гистограмма, и значения, превышающие `clip_limit`, перераспределяются равномерно:
   $$h_{\text{clipped}}(k) = \min(h(k), \text{clip\_limit})$$

3. **Локальная эквализация**: К каждому блоку применяется обычная эквализация с ограниченной гистограммой.

4. **Интерполяция**: Для пикселя на границе блоков применяется билинейная интерполяция между преобразованиями соседних блоков.

**Параметры:**
- `clip_limit`: порог ограничения контраста (обычно 2.0-4.0)
- `tile_grid_size`: размер блоков, например (8, 8)

**Преимущества перед обычной эквализацией:**
- Избегает чрезмерного усиления шума
- Адаптируется к локальным особенностям изображения
- Лучше работает с изображениями с неравномерным освещением

### Полезные функции OpenCV

- `cv2.createCLAHE(clipLimit, tileGridSize)` — создание объекта CLAHE
- `clahe.apply(channel)` — применение к одноканальному изображению
- `cv2.cvtColor()` — конвертация цветовых пространств

---

In [ ]:
def custom_clahe(image: np.ndarray, 
                 clip_limit: float = 4.0,
                 tile_grid_size: Tuple[int, int] = (8, 8)) -> np.ndarray:
    """
    Применяет CLAHE (Contrast Limited Adaptive Histogram Equalization).
    
    Args:
        image: Входное RGB изображение
        clip_limit: Порог ограничения контраста
        tile_grid_size: Размер блоков для адаптивной эквализации
    
    Returns:
        Изображение после применения CLAHE
    """
    # Конвертируем в LAB (аналог albumentations по умолчанию)
    # Создаем объект CLAHE
    # Применяем CLAHE только к L-каналу (яркость)
    # Конвертируем обратно в RGB

    # TODO    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_clahe,
    albumentations_transform=A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0),
    title="CLAHE: Сравнение реализаций",
    params={'clip_limit': 4.0, 'tile_grid_size': (8, 8)}
)

---

## 6. ColorJitter — Случайное изменение цветовых характеристик

### Описание и математика

**ColorJitter** — популярная аугментация из PyTorch/torchvision, которая случайным образом изменяет четыре цветовых характеристики изображения.

**Компоненты трансформации:**

**1. Brightness (Яркость):**
$$I_{\text{bright}} = I \times (1 + \alpha_b)$$
где $\alpha_b \in [-b, +b]$ — случайный коэффициент яркости

**2. Contrast (Контраст):**
$$I_{\text{contrast}} = (I - 128) \times (1 + \alpha_c) + 128$$
где $\alpha_c \in [-c, +c]$ — коэффициент контраста, 128 — средняя яркость

**3. Saturation (Насыщенность):**
Работает в HSV пространстве:
$$S' = S \times (1 + \alpha_s)$$
где $\alpha_s \in [-s, +s]$, $S \in [0, 255]$ — канал насыщенности

**4. Hue (Оттенок):**
Также в HSV пространстве:
$$H' = (H + \alpha_h \times 180) \mod 180$$
где $\alpha_h \in [-h, +h]$, $H \in [0, 179]$ — канал оттенка в OpenCV

**Важные детали реализации:**

1. **Порядок применения:** случайный! Каждый запуск может применять преобразования в разном порядке
2. **Пространства цветов:** 
   - Brightness, Contrast — в RGB
   - Saturation, Hue — требуют конвертации в HSV и обратно
3. **Клиппинг:** после каждого преобразования нужно ограничивать значения

**Применение:**
- Универсальная цветовая аугментация для большинства задач
- Устойчивость к различным условиям освещения
- Имитация различных камер и настроек
- Компенсация баланса белого



In [ ]:
def custom_color_jitter(image: np.ndarray,
                        brightness: float = 0.2,
                        contrast: float = 0.2,
                        saturation: float = 0.2,
                        hue: float = 0.1) -> np.ndarray:
    """
    Применяет случайное изменение яркости, контраста, насыщенности и оттенка.
    
    Args:
        image: Входное RGB изображение
        brightness: Диапазон изменения яркости [-brightness, +brightness]
        contrast: Диапазон изменения контраста
        saturation: Диапазон изменения насыщенности
        hue: Диапазон изменения оттенка
    
    Returns:
        Преобразованное изображение
    """
    result = image.copy().astype(np.float32)
    
    # TODO: 1. Brightness (Яркость)
    
    # TODO: 2. Contrast (Контраст)
    
    # TODO: 3. Saturation (Насыщенность) - работаем в HSV
   
    # TODO: 4. Hue (Оттенок)
    # Сгенерируйте alpha_h в диапазоне [-hue, +hue]
    # Измените канал H (индекс 0): hsv[:, :, 0] = (hsv[:, :, 0] + alpha_h * 180) % 180
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_color_jitter,
    albumentations_transform=A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1.0),
    title="ColorJitter: Сравнение реализаций",
    params={'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.1}
)


---

## 7. Emboss — Эффект тиснения

### Описание и математика

**Emboss (Тиснение)** — это фильтр, который создает эффект рельефной поверхности, подчеркивая границы и создавая впечатление освещения сбоку.

**Принцип работы:**

Используется свертка с ядром emboss, которое вычисляет разность между соседними пикселями:

**Ядро emboss:**
$$
K_{\text{emboss}} = \begin{bmatrix}
-2 & -1 & 0 \\
-1 & 1 & 1 \\
0 & 1 & 2
\end{bmatrix}
$$

или альтернативный вариант:
$$
K_{\text{emboss}} = \alpha \cdot \begin{bmatrix}
-1 & -1 & 0 \\
-1 & 0 & 1 \\
0 & 1 & 1
\end{bmatrix}
$$

где $\alpha$ — параметр интенсивности (обычно 1.0-2.0).

**Процесс:**
1. Конвертация в grayscale (опционально)
2. Применение свертки с ядром emboss:
   $$I_{\text{emboss}}(x, y) = \sum_{i=-1}^{1} \sum_{j=-1}^{1} K(i, j) \cdot I(x+i, y+j)$$
3. Добавление смещения для центрирования значений вокруг 128:
   $$I_{\text{final}}(x, y) = \text{clip}(I_{\text{emboss}}(x, y) + 128, 0, 255)$$

**Параметры:**
- `alpha`: интенсивность эффекта (0.0-1.0)
- `strength`: сила тиснения (1.0-2.0)

### Полезные функции OpenCV

- `cv2.filter2D(image, ddepth, kernel)` — применение произвольного фильтра (свертки)
- `cv2.cvtColor()` — конвертация цветовых пространств
- `cv2.addWeighted(src1, alpha, src2, beta, gamma)` — смешивание изображений

---

In [ ]:
def custom_emboss(image: np.ndarray, 
                  alpha: float = 1.0,
                  strength: float = 1.0) -> np.ndarray:
    """
    Применяет эффект тиснения (emboss) к изображению.
    
    Args:
        image: Входное RGB изображение
        alpha: Интенсивность смешивания с оригиналом (0.0-1.0)
        strength: Сила эффекта тиснения
    
    Returns:
        Изображение с эффектом тиснения
    """
    # Ядро для эффекта emboss
    kernel = np.array([[-1, -1, 0],
                       [-1,  0, 1],
                       [ 0,  1, 1]], dtype=np.float32) * strength

    kernel = cv2.resize(kernel, (3, 3), cv2.INTER_CUBIC)
    
    # Конвертируем в grayscale
    # Применяем фильтр emboss
    # Добавляем смещение для центрирования вокруг 128
    # Ограничиваем значения
    # Конвертируем обратно в RGB
    # Смешиваем с оригиналом по параметру alpha
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_geom_rgb,
    custom_func=custom_emboss,
    albumentations_transform=A.Emboss(alpha=(0.2, 0.5), strength=(10, 25), p=1.0),
    title="Emboss: Сравнение реализаций",
    params={'alpha': 0.35, 'strength': 0.5}
)

---

## 8. Sharpen — Повышение резкости

### Описание и математика

**Sharpen** — фильтр, усиливающий контуры и детали изображения, противоположный размытию.

**Метод 1: Convolution Kernel (Ядро свертки)**

Применяется свертка с ядром резкости:

$$K_{\text{sharpen}} = \begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}$$

или более сильное:

$$K_{\text{strong}} = \begin{bmatrix}
-1 & -1 & -1 \\
-1 & 9 & -1 \\
-1 & -1 & -1
\end{bmatrix}$$

**Метод 2: Unsharp Masking (Нерезкая маска)**

Более контролируемый подход:

$$I_{\text{sharp}} = I + \alpha \cdot (I - G_\sigma * I)$$

где:
- $I$ — исходное изображение
- $G_\sigma * I$ — размытое изображение (Gaussian blur)
- $(I - G_\sigma * I)$ — высокочастотные детали (маска)
- $\alpha$ — коэффициент усиления резкости (обычно 0.5-2.0)

**Алгоритм Unsharp Masking:**

1. Размыть изображение гауссовым фильтром: $I_{\text{blur}} = G_\sigma * I$
2. Вычислить маску деталей: $M = I - I_{\text{blur}}$
3. Добавить маску к оригиналу: $I_{\text{sharp}} = I + \alpha \cdot M$
4. Clip значения в диапазон [0, 255]

**Параметры:**
- `alpha` — сила эффекта (0.0 = нет эффекта, 1.0 = полный)
- `lightness` — яркость резкости (влияет на финальную яркость)
- `blur_sigma` — sigma для Gaussian blur (обычно 1-3)

**Применение:**
- Улучшение четкости изображений
- Подчеркивание контуров объектов
- Коррекция слегка размытых фотографий
- Аугментация для детекции мелких объектов



In [ ]:
def custom_sharpen(image: np.ndarray,
                   alpha: float = 0.5,
                   lightness: float = 1.0,
                   blur_sigma: float = 1.0) -> np.ndarray:
    """
    Применяет повышение резкости методом Unsharp Masking.
    
    Args:
        image: Входное изображение
        alpha: Сила эффекта резкости (0.0-2.0)
        lightness: Коэффициент яркости
        blur_sigma: Sigma для Gaussian blur
    
    Returns:
        Изображение с повышенной резкостью
    """
    # TODO: Реализуйте Unsharp Masking
    
    # 1. Конвертируйте изображение в float для точных вычислений
    img_float = image.astype(np.float32)
    
    # 2. Примените Gaussian Blur
    # Вычислите размер ядра: ksize должен быть нечетным
    # Правило: ksize = int(6 * sigma + 1) | 1  (побитовое ИЛИ для нечетности)
    # Примените cv2.GaussianBlur
      
    # 3. Вычислите маску деталей: mask = image - blurred
    # 4. Добавьте маску к оригиналу с весом alpha
    # result = image + alpha * mask
    
    # 5. Примените lightness (яркость)
    # 6. Подготовьте результат
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_sharpen,
    albumentations_transform=A.Sharpen(alpha=(0.5, 0.5), lightness=(1.0, 1.0), p=1.0),
    title="Sharpen: Сравнение реализаций",
    params={'alpha': 0.5, 'lightness': 1.0, 'blur_sigma': 1.0}
)


---

## 9. GaussianBlur — Размытие по Гауссу

### Описание и математика

**Gaussian Blur** — это классический фильтр размытия, использующий двумерную функцию Гаусса в качестве ядра свертки.

**Двумерная функция Гаусса:**
$$
G(x, y) = \frac{1}{2\pi\sigma^2} \exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right)
$$

где:
- $\sigma$ — стандартное отклонение (параметр размытия)
- $x, y$ — расстояния от центра ядра

**Ядро Гаусса** размером $(2k+1) \times (2k+1)$:
$$
K(i, j) = G(i - k, j - k)
$$

Ядро нормализуется так, чтобы сумма всех элементов была равна 1:
$$
K_{\text{normalized}} = \frac{K}{\sum_{i,j} K(i, j)}
$$

**Применение свертки:**
$$
I_{\text{blur}}(x, y) = \sum_{i=-k}^{k} \sum_{j=-k}^{k} K(i, j) \cdot I(x+i, y+j)
$$

**Параметры:**
- `blur_limit`: максимальный размер ядра (должен быть нечетным)
- `sigma_limit`: диапазон значений $\sigma$

**Применение:**
- Уменьшение шума
- Сглаживание изображений
- Имитация расфокусировки камеры

### Два подхода реализации

**1. Через генерацию ядра и применение свертки:**
- Создаем ядро Гаусса вручную
- Применяем через `cv2.filter2D()`

**2. Через готовый метод OpenCV:**
- Используем `cv2.GaussianBlur()` напрямую

### Полезные функции OpenCV

- `cv2.getGaussianKernel(ksize, sigma)` — генерация одномерного ядра Гаусса
- `cv2.GaussianBlur(image, ksize, sigmaX, sigmaY)` — готовая функция размытия
- `cv2.filter2D(image, ddepth, kernel)` — применение произвольной свертки

---

In [ ]:
def custom_gaussian_blur_manual(image: np.ndarray, 
                                kernel_size: int = 7,
                                sigma: float = 0) -> np.ndarray:
    """
    Применяет Gaussian Blur через генерацию ядра и применение свертки.
    
    Args:
        image: Входное RGB изображение
        kernel_size: Размер ядра (должен быть нечетным)
        sigma: Стандартное отклонение. Если 0, вычисляется автоматически
    
    Returns:
        Размытое изображение
    """
    # Убеждаемся, что kernel_size нечетный
    if kernel_size % 2 == 0:
        kernel_size += 1
    
    # Если sigma=0, вычисляем автоматически
    if sigma == 0:
        sigma = 0.3 * ((kernel_size - 1) * 0.5 - 1) + 0.8
    
    # TODO
    return result


def custom_gaussian_blur_cv2(image: np.ndarray, 
                             kernel_size: int = 7,
                             sigma: float = 0) -> np.ndarray:
    """
    Применяет Gaussian Blur через готовый метод cv2.GaussianBlur().
    
    Args:
        image: Входное RGB изображение
        kernel_size: Размер ядра (должен быть нечетным)
        sigma: Стандартное отклонение
    
    Returns:
        Размытое изображение
    """
    # Убеждаемся, что kernel_size нечетный
    if kernel_size % 2 == 0:
        kernel_size += 1
    
    # TODO
    return result


# Визуализация сравнения для ручной реализации
print("Вариант 1: Через генерацию ядра и свертку")
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_gaussian_blur_manual,
    albumentations_transform=A.GaussianBlur(blur_limit=(7, 7), sigma_limit=(1.5, 1.5), p=1.0),
    title="GaussianBlur (Manual): Сравнение реализаций",
    params={'kernel_size': 7, 'sigma': 1.5}
)

# Визуализация сравнения для cv2.GaussianBlur
print("\nВариант 2: Через готовый метод cv2.GaussianBlur()")
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_gaussian_blur_cv2,
    albumentations_transform=A.GaussianBlur(blur_limit=(7, 7), sigma_limit=(1.5, 1.5), p=1.0),
    title="GaussianBlur (CV2): Сравнение реализаций",
    params={'kernel_size': 7, 'sigma': 1.5}
)

---

## 10. MotionBlur — Размытие движения

### Описание и математика

**Motion Blur** имитирует размытие, возникающее при движении камеры или объекта во время съемки. Эффект создается путем применения направленного размытия.

**Принцип работы:**

Создается линейное ядро свертки, ориентированное под определенным углом:

1. **Создание линейного ядра**: Ядро представляет собой линию длиной $k$ пикселей:
   $$K_{\text{line}} = \text{единицы вдоль линии, нули в остальных местах}$$

2. **Нормализация**:
   $$K_{\text{normalized}} = \frac{K_{\text{line}}}{\sum K_{\text{line}}}$$

3. **Поворот ядра**: Ядро поворачивается на угол $\theta$ с помощью матрицы вращения:
   $$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix}$$

4. **Применение свертки**:
   $$I_{\text{motion}}(x, y) = \sum_{i,j} K(i, j) \cdot I(x+i, y+j)$$

**Параметры:**
- `blur_limit`: размер ядра (длина траектории движения)
- `allow_shifted`: разрешить ли смещение центра ядра

**Применение:**
- Аугментация для имитации движущихся объектов
- Симуляция низкой выдержки камеры
- Создание эффекта скорости

### Полезные функции OpenCV

- `cv2.getRotationMatrix2D(center, angle, scale)` — создание матрицы поворота
- `cv2.warpAffine(image, M, dsize)` — применение аффинного преобразования
- `cv2.filter2D(image, ddepth, kernel)` — применение свертки

---

In [ ]:
def custom_motion_blur(image: np.ndarray, 
                       kernel_size: int = 15,
                       angle: float = 45) -> np.ndarray:
    """
    Применяет размытие движения (motion blur).
    
    Args:
        image: Входное RGB изображение
        kernel_size: Размер ядра (длина траектории движения)
        angle: Угол направления движения в градусах
    
    Returns:
        Изображение с размытием движения
    """
    # Создаем горизонтальное линейное ядро
    kernel = np.zeros((kernel_size, kernel_size), dtype=np.float32)
    kernel[kernel_size // 2, :] = 1.0
    
    # Поворачиваем ядро на заданный угол
    # Нормализуем после поворота (из-за интерполяции)
    # Применяем свертку
    # TODO
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_motion_blur,
    albumentations_transform=A.MotionBlur(blur_limit=(15, 15), p=1.0),
    title="MotionBlur: Сравнение реализаций",
    params={'kernel_size': 15, 'angle': np.random.randint(0, 360)}
)

---

## 11. ZoomBlur — Размытие при приближении

### Описание и математика

**ZoomBlur** — имитирует размытие, возникающее при быстром зумировании (приближении/удалении) камеры или при движении камеры вперед/назад к объекту.

**Математический процесс:**

Изображение создается как взвешенное среднее нескольких масштабированных версий:

$$I_{\text{zoom}}(x, y) = \frac{1}{N} \sum_{i=1}^{N} I_{\text{scaled}}^{(i)}(x, y)$$

где каждая копия $I_{\text{scaled}}^{(i)}$ масштабирована с коэффициентом:

$$s_i = 1 + \frac{i-1}{N-1} \cdot (s_{\max} - 1)$$

**Алгоритм:**

1. Определить количество шагов: $N = \lceil \frac{s_{\max} - 1}{\text{step}} \rceil$
2. Для каждого $i$ от 1 до $N$:
   - Вычислить коэффициент масштабирования $s_i$
   - Масштабировать изображение относительно центра
   - Добавить к накопителю
3. Усреднить все копии

**Масштабирование относительно центра:**

Для масштабирования относительно центра изображения нужно:
1. Вычислить размеры: $W_{new} = W \cdot s$, $H_{new} = H \cdot s$
2. Масштабировать изображение
3. Если $s > 1$ (увеличение): вырезать центральную часть размера $(W, H)$
4. Если $s < 1$ (уменьшение): добавить padding

**Центр crop/padding:**
$$x_{\text{start}} = \frac{W_{new} - W}{2}, \quad y_{\text{start}} = \frac{H_{new} - H}{2}$$

**Применение:**
- Имитация эффекта зумирования камеры
- Устойчивость к размытию при движении
- Художественные эффекты для видео
- Эмуляция изменения фокусного расстояния



In [ ]:
def custom_zoom_blur(image: np.ndarray,
                     max_factor: float = 1.31,
                     step_factor: float = 0.01) -> np.ndarray:
    """
    Применяет zoom blur — размытие при зумировании.
    
    Args:
        image: Входное изображение
        max_factor: Максимальный коэффициент зума (>1.0)
        step_factor: Шаг изменения зума
    
    Returns:
        Изображение с zoom blur эффектом
    """
    height, width = image.shape[:2]
    center_y, center_x = height // 2, width // 2
    
    # TODO: 1. Вычислите количество шагов
    # num_steps = int((max_factor - 1.0) / step_factor)
    num_steps = max(int((max_factor - 1.0) / step_factor), 1)
    
    # Инициализируем накопитель
    result = np.zeros_like(image, dtype=np.float32)
    # TODO: 2. Генерируем масштабированные копии
    for i in range(num_steps):
        # Вычислите текущий коэффициент масштабирования
        # Вычислите новые размеры
        # Масштабируйте изображение
       
        # TODO: 3. Если увеличили (scale > 1), вырежьте центральную часть
        if scale >= 1.0:
            # Вычислите координаты для вырезания центра
        else:
            # Если уменьшили, добавьте padding (редкий случай при max_factor > 1)
            
        # Добавляем к результату
    
    # TODO: 4. Усредните результат
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_zoom_blur,
    albumentations_transform=A.ZoomBlur(max_factor=1.31, step_factor=0.01, p=1.0),
    title="ZoomBlur: Сравнение реализаций",
    params={'max_factor': 1.31, 'step_factor': 0.01}
)


---

## 12. ISONoise — ISO шум

### Описание и математика

**ISO Noise** имитирует шум, возникающий в камере при высоких значениях ISO (чувствительности матрицы). Этот шум характеризуется как **цветовым шумом** (color noise), так и **яркостным шумом** (luminance noise).

**Модель шума:**

Шум камеры состоит из двух компонентов:

1. **Яркостный шум (Luminance noise)**: добавляется к каналу яркости
   $$L_{\text{noisy}} = L + \mathcal{N}(0, \sigma_L^2)$$

2. **Цветовой шум (Color noise)**: добавляется к цветовым каналам
   $$C_{\text{noisy}} = C + \mathcal{N}(0, \sigma_C^2)$$

где $\mathcal{N}(0, \sigma^2)$ — нормальное распределение с нулевым средним и дисперсией $\sigma^2$.

**Алгоритм:**

1. Конвертация RGB → HSV или LAB
2. Добавление гауссовского шума к каналу яркости (V или L):
   $$V_{\text{new}} = V + \text{Gauss}(0, \text{intensity\_luma})$$
3. Добавление гауссовского шума к цветовым каналам (H, S или A, B):
   $$S_{\text{new}} = S + \text{Gauss}(0, \text{intensity\_color})$$
4. Ограничение значений и конвертация обратно в RGB

**Параметры:**
- `color_shift`: диапазон цветового сдвига (0.01-0.05)
- `intensity`: общая интенсивность шума (0.1-0.5)

**Применение:**
- Симуляция съемки в условиях низкой освещенности
- Повышение робастности модели к зашумленным данным
- Реалистичная аугментация для задач распознавания в сложных условиях

### Полезные функции OpenCV

- `np.random.normal(mean, std, size)` — генерация гауссовского шума
- `cv2.cvtColor(image, cv2.COLOR_RGB2HSV)` — конвертация в HSV
- `cv2.cvtColor(image, cv2.COLOR_HSV2RGB)` — обратная конвертация

---

In [ ]:
def custom_iso_noise(image: np.ndarray, 
                     color_shift: float = 0.05,
                     intensity: float = 0.5) -> np.ndarray:
    """
    Добавляет ISO шум (имитация высокого ISO камеры).
    
    Args:
        image: Входное RGB изображение
        color_shift: Интенсивность цветового шума
        intensity: Общая интенсивность шума
    
    Returns:
        Зашумленное изображение
    """
    # Конвертируем в HSV
    
    # Генерируем гауссовский шум для каждого канала
    # Hue (оттенок) - цветовой шум
    # hue_noise = ??
    
    # Saturation (насыщенность) - цветовой шум
    # sat_noise = ??
    
    # Value (яркость) - яркостный шум
    # val_noise = ??
    
    # Добавляем шум к каждому каналу
   
    # Конвертируем обратно

    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_iso_noise,
    albumentations_transform=A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1.0),
    title="ISONoise: Сравнение реализаций",
    params={'color_shift': 0.03, 'intensity': 0.3}
)

---

## 13. Spatter — Брызги и пятна

### Описание и математика

**Spatter** — имитирует брызги воды, грязи, краски или других жидкостей на объективе камеры или поверхности изображения.

**Математическая модель:**

Генерируется маска пятен $M(x, y) \in [0, 1]$, которая применяется к изображению:

**Режим Rain (дождь):**
$$I'(x, y) = (1 - \alpha \cdot M(x, y)) \cdot I(x, y) + \alpha \cdot M(x, y) \cdot I_{\text{blur}}(x, y)$$

**Режим Mud (грязь):**
$$I'(x, y) = (1 - \alpha \cdot M(x, y)) \cdot I(x, y) + \alpha \cdot M(x, y) \cdot C_{\text{mud}}$$

где:
- $M(x, y)$ — маска пятен
- $\alpha$ — интенсивность эффекта
- $I_{\text{blur}}$ — размытая версия изображения (для капель воды)
- $C_{\text{mud}}$ — цвет грязи (обычно темно-коричневый)

**Генерация маски пятен:**

1. **Создание noise карты:**
   $$N(x, y) \sim \mathcal{N}(\mu, \sigma^2)$$
   где $\mu$ — mean, $\sigma$ — std параметры

2. **Пороговая фильтрация:**
   $$M_{\text{binary}}(x, y) = \begin{cases} 
   1, & N(x, y) > \text{threshold} \\
   0, & \text{otherwise}
   \end{cases}$$

3. **Размытие маски (для плавных границ):**
   $$M(x, y) = G_{\sigma} * M_{\text{binary}}(x, y)$$
   где $G_{\sigma}$ — гауссово ядро с параметром `gauss_sigma`

**Параметры:**
- `mean` — среднее значение для генерации noise (обычно 0.65)
- `std` — стандартное отклонение (обычно 0.3)
- `gauss_sigma` — сглаживание границ пятен
- `cutout_threshold` — порог отсечения для маски
- `intensity` — сила эффекта
- `mode` — 'rain' или 'mud'

**Применение:**
- Имитация погодных условий (дождь, снег)
- Устойчивость к загрязнениям камеры
- Реалистичные данные для автономных систем
- Off-road сценарии для автомобилей



In [ ]:
def custom_spatter(image: np.ndarray,
                   mean: float = 0.65,
                   std: float = 0.3,
                   gauss_sigma: float = 2.0,
                   cutout_threshold: float = 0.68,
                   intensity: float = 0.6,
                   mode: str = 'rain') -> np.ndarray:
    """
    Применяет эффект брызг (дождь или грязь).
    
    Args:
        image: Входное изображение
        mean: Среднее для генерации noise
        std: Стандартное отклонение
        gauss_sigma: Sigma для размытия маски
        cutout_threshold: Порог для отсечения маски
        intensity: Интенсивность эффекта (0-1)
        mode: 'rain' или 'mud'
    
    Returns:
        Изображение с эффектом брызг
    """
    height, width = image.shape[:2]
    
    # TODO: 1. Генерируем noise карту с нормальным распределением  
    # TODO: 2. Создаем бинарную маску по порогу
    # TODO: 3. Размываем маску для плавных границ
    # Вычислите ksize для GaussianBlur
    # ksize = int(6 * gauss_sigma + 1) | 1
    # mask_smooth = cv2.GaussianBlur(mask_binary, (ksize, ksize), gauss_sigma)
    
    # Нормализуем маску в диапазон [0, 1]
       
    # Применяем интенсивность
    # Расширяем маску до 3 каналов
    
    # TODO: 4. Применяем эффект в зависимости от режима
    if mode == 'rain':
        # Для дождя: смешиваем с размытой версией
        blurred = cv2.GaussianBlur(image, (15, 15), 0)
        result = (1 - mask_3ch) * img_float + mask_3ch * blurred.astype(np.float32)
    else:  # mode == 'mud'
        # Для грязи: смешиваем с темным цветом
        mud_color = np.array([80, 70, 60], dtype=np.float32)  # Темно-коричневый
        result = (1 - mask_3ch) * img_float + mask_3ch * mud_color

    # todo
    return result 
    
# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_spatter,
    albumentations_transform=A.Spatter(mean=1, std=1, gauss_sigma=20, intensity=1, mode='rain', p=1.0),
    title="Spatter (Rain): Сравнение реализаций",
    params={'mean': 0.65, 'std': 0.3, 'gauss_sigma': 2.0, 'cutout_threshold': 0.68, 'intensity': 0.6, 'mode': 'rain'}
)


---

## 14. ChromaticAberration — Хроматическая аберрация

### Описание и математика

**Chromatic Aberration (Хроматическая аберрация)** — это оптический дефект, при котором разные длины волн света (разные цвета) фокусируются в разных точках. В результате на краях объектов появляются цветные ореолы.

**Физическая природа:**
- Показатель преломления линзы зависит от длины волны света (дисперсия)
- Синий свет преломляется сильнее, чем красный
- Различные цветовые каналы смещаются радиально от центра изображения

**Математическая модель:**

Для каждого канала $c \in \{R, G, B\}$ применяется радиальное смещение:

$$
\begin{aligned}
r &= \sqrt{(x - c_x)^2 + (y - c_y)^2} \\
\text{scale}_c &= 1 + k_c \cdot r^2 \\
x'_c &= c_x + (x - c_x) \cdot \text{scale}_c \\
y'_c &= c_y + (y - c_y) \cdot \text{scale}_c
\end{aligned}
$$

где:
- $(c_x, c_y)$ — центр изображения
- $r$ — расстояние от центра
- $k_c$ — коэффициент искажения для канала $c$ (обычно $k_R < 0 < k_B$)

**Алгоритм:**
1. Разделение на RGB каналы
2. Для каждого канала создание карты смещения
3. Применение `cv2.remap()` для каждого канала с разными параметрами
4. Объединение каналов обратно

**Параметры:**
- `primary_distortion_limit`: коэффициент основного искажения (0.01-0.05)
- `secondary_distortion_limit`: коэффициент вторичного искажения (0.01-0.05)
- `mode`: тип аберрации ('green_purple', 'red_blue', 'random')

### Полезные функции OpenCV

- `cv2.remap(image, map_x, map_y, interpolation)` — применение произвольного отображения координат
- `np.meshgrid(x, y)` — создание сетки координат
- `cv2.split(image)` и `cv2.merge(channels)` — разделение и объединение каналов

---

In [ ]:
def custom_chromatic_aberration(image: np.ndarray, 
                                primary_distortion: float = 0.02,
                                secondary_distortion: float = 0.01,
                                mode: str = 'green_purple') -> np.ndarray:
    """
    Применяет эффект хроматической аберрации.
    
    Args:
        image: Входное RGB изображение
        primary_distortion: Основной коэффициент искажения
        secondary_distortion: Вторичный коэффициент искажения
        mode: Тип аберрации ('green_purple', 'red_blue', 'random')
    
    Returns:
        Изображение с хроматической аберрацией
    """
    h, w = image.shape[:2]
    
    # Центр изображения
    cx, cy = w / 2, h / 2
    
    # Создаем сетку координат
    # Разделяем каналы
    
    # Определяем коэффициенты искажения для каждого канала
    if mode == 'green_purple':
        distortions = [primary_distortion, -secondary_distortion, -primary_distortion]
    elif mode == 'red_blue':
        distortions = [primary_distortion, 0, -primary_distortion]
    else:  # random
        distortions = [
            np.random.uniform(-primary_distortion, primary_distortion),
            np.random.uniform(-secondary_distortion, secondary_distortion),
            np.random.uniform(-primary_distortion, primary_distortion)
        ]
    
    # Применяем искажение к каждому каналу
    result_channels = []
    for i, (channel, k) in enumerate(zip(channels, distortions)):
        # Вычисляем смещение
        # scale = ??
        
        # Новые координаты
        # map_x = ??
        # map_y = ??
        
        # Применяем remap
        # remapped = cv2.remap(channel, map_x, map_y, 
        #                     interpolation=cv2.INTER_LINEAR,
        #                     borderMode=cv2.BORDER_REFLECT)
        # result_channels.append(remapped)
    
    # Объединяем каналы
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_chromatic_aberration,
    albumentations_transform=A.ChromaticAberration(
        primary_distortion_limit=0.2, 
        secondary_distortion_limit=0.1, 
        mode='green_purple',
        p=1.0
    ),
    title="ChromaticAberration: Сравнение реализаций",
    params={'primary_distortion': 0.2, 'secondary_distortion': 0.1, 'mode': 'green_purple'}
)

---

## 15. GlassBlur — Размытие стеклом

### Описание и математика

**Glass Blur** имитирует эффект просмотра изображения через стекло со случайными неровностями. Создается локальное случайное искажение пикселей.

**Принцип работы:**

Для каждого пикселя $(x, y)$ выполняется случайное смещение в окрестности:

$$
\begin{aligned}
\Delta x &\sim \mathcal{U}(-\text{max\_delta}, +\text{max\_delta}) \\
\Delta y &\sim \mathcal{U}(-\text{max\_delta}, +\text{max\_delta}) \\
x' &= x + \Delta x \\
y' &= y + \Delta y \\
I_{\text{new}}(x, y) &= I_{\text{old}}(x', y')
\end{aligned}
$$

где $\mathcal{U}(a, b)$ — равномерное распределение на отрезке $[a, b]$.

**Алгоритм:**
1. Создание карты случайных смещений для каждого пикселя
2. Применение размытия к картам смещений (для плавности)
3. Использование `cv2.remap()` для применения искажений
4. Опционально: применение дополнительного размытия для сглаживания

**Параметры:**
- `sigma`: стандартное отклонение для размытия карты смещений (0.5-2.0)
- `max_delta`: максимальное смещение пикселей (1-4)
- `iterations`: количество итераций (1-2)

**Применение:**
- Имитация съемки через стекло или воду
- Создание эффекта искажения
- Аугментация для повышения робастности

### Полезные функции OpenCV

- `np.random.uniform(low, high, size)` — генерация случайных смещений
- `cv2.GaussianBlur()` — размытие карт смещений
- `cv2.remap(image, map_x, map_y, interpolation)` — применение искажений

---

In [ ]:
def custom_glass_blur(image: np.ndarray, 
                      sigma: float = 0.7,
                      max_delta: int = 2,
                      iterations: int = 1) -> np.ndarray:
    """
    Применяет эффект размытия стеклом (glass blur).
    
    Args:
        image: Входное RGB изображение
        sigma: Стандартное отклонение для размытия карты смещений
        max_delta: Максимальное случайное смещение пикселей
        iterations: Количество итераций применения эффекта
    
    Returns:
        Изображение с эффектом glass blur
    """
    h, w = image.shape[:2]
    result = image.copy()
    
    for _ in range(iterations):
        # Создаем случайные смещения
        # delta_x = ??
        # delta_y = ??
        
        # Размываем карты смещений для плавности
        if sigma > 0:
            ksize = int(sigma * 3) * 2 + 1  # размер ядра
            # TODO
        
        # Создаем карты координат
        # map_x = ??
        # map_y = ??
        
        # Применяем remap
        result = cv2.remap(result, map_x, map_y, 
                          interpolation=cv2.INTER_LINEAR,
                          borderMode=cv2.BORDER_REFLECT)
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_glass_blur,
    albumentations_transform=A.GlassBlur(sigma=0.7, max_delta=2, iterations=1, p=1.0),
    title="GlassBlur: Сравнение реализаций",
    params={'sigma': 0.7, 'max_delta': 2, 'iterations': 1}
)

---

## 16. FancyPCA — Продвинутая PCA аугментация

### Описание и математика

**Fancy PCA** — это изощренная цветовая аугментация, предложенная в статье AlexNet (Krizhevsky et al., 2012). Она изменяет RGB значения вдоль главных компонент цветовых вариаций в наборе данных.

**Идея:**
- Находим главные компоненты цветовых вариаций в изображении
- Добавляем возмущение вдоль этих компонент
- Сохраняем естественность цветов (в отличие от простого RGB shift)

**Математический алгоритм:**

1. **Подготовка данных**: Для каждого пикселя получаем RGB вектор $\mathbf{x} = [R, G, B]^T$

2. **Вычисление ковариационной матрицы** всех пикселей изображения:
   $$\mathbf{C} = \frac{1}{N} \sum_{i=1}^{N} (\mathbf{x}_i - \bar{\mathbf{x}})(\mathbf{x}_i - \bar{\mathbf{x}})^T$$
   где $\bar{\mathbf{x}}$ — среднее по всем пикселям.

3. **Собственное разложение** ковариационной матрицы:
   $$\mathbf{C} = \mathbf{P} \mathbf{\Lambda} \mathbf{P}^T$$
   где $\mathbf{P} = [\mathbf{p}_1, \mathbf{p}_2, \mathbf{p}_3]$ — собственные векторы (главные компоненты),
   $\mathbf{\Lambda} = \text{diag}(\lambda_1, \lambda_2, \lambda_3)$ — собственные значения.

4. **Генерация возмущения**: Создаем случайный вектор
   $$\boldsymbol{\alpha} = [\alpha_1, \alpha_2, \alpha_3]^T, \quad \alpha_i \sim \mathcal{N}(0, \sigma^2)$$

5. **Добавление возмущения** к каждому пикселю:
   $$\mathbf{x}_{\text{new}} = \mathbf{x}_{\text{old}} + \mathbf{P} \mathbf{\Lambda}^{1/2} \boldsymbol{\alpha}$$
   
   или эквивалентно:
   $$\mathbf{x}_{\text{new}} = \mathbf{x}_{\text{old}} + \sum_{i=1}^{3} \alpha_i \sqrt{\lambda_i} \mathbf{p}_i$$

**Параметры:**
- `alpha`: стандартное отклонение для случайных $\alpha_i$ (обычно 0.1)

**Применение:**
- Сохраняет естественные цветовые соотношения
- Более реалистичная цветовая аугментация
- Использовалась в AlexNet для победы в ImageNet 2012

### Полезные функции OpenCV/NumPy

- `np.cov(data, rowvar=False)` — вычисление ковариационной матрицы
- `np.linalg.eig(matrix)` — собственное разложение
- `np.random.normal(mean, std, size)` — генерация нормально распределенных чисел

---

In [ ]:
def custom_fancy_pca(image: np.ndarray, alpha: float = 0.1) -> np.ndarray:
    """
    Применяет Fancy PCA аугментацию (из AlexNet paper).
    
    Args:
        image: Входное RGB изображение
        alpha: Стандартное отклонение для случайных коэффициентов
    
    Returns:
        Изображение с примененной Fancy PCA
    """
    # TODO
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_fancy_pca,
    albumentations_transform=A.FancyPCA(alpha=0.1, p=1.0),
    title="FancyPCA: Сравнение реализаций",
    params={'alpha': 0.1}
)

---

## 17. Rotate — Поворот изображения

### Описание и математика

**Rotate** — базовая геометрическая трансформация, поворачивающая изображение вокруг центра на заданный угол.

**Математика поворота:**

Матрица поворота на угол $\theta$ (против часовой стрелки):
$$
\mathbf{R}(\theta) = \begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}
$$

Для поворота вокруг произвольного центра $(c_x, c_y)$:

1. **Сдвиг** к началу координат:
   $$\mathbf{x}' = \mathbf{x} - \mathbf{c}$$

2. **Поворот**:
   $$\mathbf{x}'' = \mathbf{R}(\theta) \mathbf{x}'$$

3. **Сдвиг** обратно:
   $$\mathbf{x}_{\text{new}} = \mathbf{x}'' + \mathbf{c}$$

**Аффинная матрица преобразования:**
$$
\mathbf{M} = \begin{bmatrix}
\cos\theta & -\sin\theta & (1-\cos\theta)c_x + \sin\theta \cdot c_y \\
\sin\theta & \cos\theta & (1-\cos\theta)c_y - \sin\theta \cdot c_x
\end{bmatrix}
$$

**Параметры:**
- `limit`: диапазон углов поворота в градусах (например, (-45, 45))
- `interpolation`: метод интерполяции (LINEAR, CUBIC, NEAREST)
- `border_mode`: режим обработки границ (REFLECT, CONSTANT, REPLICATE)

**Применение:**
- Базовая аугментация для инвариантности к повороту
- Исправление ориентации изображений
- Создание вариаций для обучения

### Полезные функции OpenCV

- `cv2.getRotationMatrix2D(center, angle, scale)` — создание матрицы поворота
- `cv2.warpAffine(image, M, dsize, flags, borderMode)` — применение аффинного преобразования

---

In [ ]:
def custom_rotate(image: np.ndarray, 
                  angle: float = 0,
                  interpolation: int = cv2.INTER_LINEAR,
                  border_mode: int = cv2.BORDER_REFLECT_101) -> np.ndarray:
    """
    Поворачивает изображение на заданный угол.
    
    Args:
        image: Входное RGB изображение
        angle: Угол поворота в градусах (положительный = против часовой)
        interpolation: Метод интерполяции
        border_mode: Режим обработки границ
    
    Returns:
        Повернутое изображение
    """
    # TODO
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_geom_rgb,
    custom_func=custom_rotate,
    albumentations_transform=A.Rotate(limit=(-30, 30), p=1.0),
    title="Rotate: Сравнение реализаций",
    params={'angle': np.random.uniform(-30, 30)}
)

---

## 18. Perspective — Перспективная трансформация

### Описание и математика

**Perspective** — геометрическая трансформация, изменяющая перспективу изображения, как будто оно снято под другим углом или с другой точки обзора.

**Математическая основа — Проективное преобразование (Гомография):**

Каждая точка $(x, y)$ трансформируется в $(x', y')$ через матрицу гомографии $H$:

$$\begin{bmatrix} wx' \\ wy' \\ w \end{bmatrix} = \begin{bmatrix}
h_{11} & h_{12} & h_{13} \\
h_{21} & h_{22} & h_{23} \\
h_{31} & h_{32} & h_{33}
\end{bmatrix} \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}$$

Финальные координаты (после деления на $w$):

$$x' = \frac{h_{11}x + h_{12}y + h_{13}}{h_{31}x + h_{32}y + h_{33}}, \quad y' = \frac{h_{21}x + h_{22}y + h_{23}}{h_{31}x + h_{32}y + h_{33}}$$

**Вычисление матрицы гомографии:**

Для определения $H$ нужны 4 пары соответствующих точек:

**Исходные точки (углы изображения):**
$$P_{\text{src}} = \{(0, 0), (W, 0), (W, H), (0, H)\}$$

**Целевые точки (со случайным смещением):**
$$P_{\text{dst}} = \{(x_i + \Delta x_i, y_i + \Delta y_i)\}_{i=1}^{4}$$

где $\Delta x_i, \Delta y_i$ — случайные смещения:
$$\Delta x_i \sim U(-s \cdot W, +s \cdot W), \quad \Delta y_i \sim U(-s \cdot H, +s \cdot H)$$

$s$ — параметр `scale` (обычно 0.05-0.15)

**Алгоритм:**

1. Определить 4 угловые точки изображения
2. Сгенерировать случайные смещения для каждого угла
3. Вычислить матрицу гомографии: $H = \text{getPerspectiveTransform}(P_{\text{src}}, P_{\text{dst}})$
4. Применить трансформацию: $I' = \text{warpPerspective}(I, H, (W, H))$

**Свойства гомографии:**
- Прямые линии остаются прямыми
- Параллельные линии могут пересекаться (точка схода)
- Сохраняется коллинеарность точек
- НЕ сохраняются углы и расстояния

**Применение:**
- Имитация съемки под разными углами
- Аугментация для OCR (распознавание текста на фото документов)
- Детекция объектов с различных ракурсов
- Коррекция перспективных искажений



In [ ]:
def custom_perspective(image: np.ndarray,
                       scale: float = 0.05) -> np.ndarray:
    """
    Применяет перспективную трансформацию.
    
    Args:
        image: Входное изображение
        scale: Масштаб смещения углов (доля от размера изображения)
    
    Returns:
        Трансформированное изображение
    """
    height, width = image.shape[:2]
    
    # TODO
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_color_rgb,
    custom_func=custom_perspective,
    albumentations_transform=A.Perspective(scale=(0.05, 0.05), p=1.0),
    title="Perspective: Сравнение реализаций",
    params={'scale': 0.05}
)


---

## 19. RandomResizedCrop — Случайный кроп с изменением размера

### Описание и математика

**RandomResizedCrop** — популярная аугментация, которая:
1. Вырезает случайную область из изображения
2. Изменяет размер вырезанной области до целевого размера

Это комбинация **random crop** и **resize**, часто используется в обучении классификационных моделей (например, ResNet, EfficientNet).

**Алгоритм:**

1. **Выбор масштаба площади**:
   $$\text{area\_scale} \sim \mathcal{U}(\text{scale\_min}, \text{scale\_max})$$
   $$\text{target\_area} = \text{area\_scale} \cdot W \cdot H$$

2. **Выбор соотношения сторон**:
   $$\text{aspect\_ratio} \sim \mathcal{U}(\text{ratio\_min}, \text{ratio\_max})$$

3. **Вычисление размеров кропа**:
   $$w_{\text{crop}} = \sqrt{\text{target\_area} \cdot \text{aspect\_ratio}}$$
   $$h_{\text{crop}} = \sqrt{\text{target\_area} / \text{aspect\_ratio}}$$

4. **Случайная позиция кропа**:
   $$x \sim \mathcal{U}(0, W - w_{\text{crop}})$$
   $$y \sim \mathcal{U}(0, H - h_{\text{crop}})$$

5. **Вырезание и resize**:
   $$I_{\text{crop}} = I[y:y+h_{\text{crop}}, x:x+w_{\text{crop}}]$$
   $$I_{\text{result}} = \text{resize}(I_{\text{crop}}, (\text{height}, \text{width}))$$

**Параметры:**
- `height, width`: целевой размер выходного изображения
- `scale`: диапазон масштаба площади (например, (0.08, 1.0))
- `ratio`: диапазон соотношения сторон (например, (0.75, 1.33))

**Применение:**
- Основная аугментация в ImageNet training
- Создание масштабной и позиционной инвариантности
- Увеличение разнообразия данных

### Полезные функции OpenCV

- `cv2.resize(image, dsize, interpolation)` — изменение размера
- Простая индексация NumPy для кропа: `image[y1:y2, x1:x2]`
- `np.random.uniform(low, high)` — генерация случайных параметров

---

In [ ]:
def custom_random_resized_crop(image: np.ndarray, 
                               height: int = 224,
                               width: int = 224,
                               scale: Tuple[float, float] = (0.08, 1.0),
                               ratio: Tuple[float, float] = (0.75, 1.33)) -> np.ndarray:
    """
    Применяет random resized crop.
    
    Args:
        image: Входное RGB изображение
        height: Высота выходного изображения
        width: Ширина выходного изображения
        scale: Диапазон масштаба площади (min, max)
        ratio: Диапазон соотношения сторон (min, max)
    
    Returns:
        Обрезанное и измененное изображение
    """
    # TODO
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_geom_rgb,
    custom_func=custom_random_resized_crop,
    albumentations_transform=A.RandomResizedCrop(
        size=(img_geom_rgb.shape[0], img_geom_rgb.shape[1]),
        scale=(0.5, 1.0),
        ratio=(0.75, 1.33),
        p=1.0
    ),
    title="RandomResizedCrop: Сравнение реализаций",
    params={
        'height': img_geom_rgb.shape[0],
        'width': img_geom_rgb.shape[1],
        'scale': (0.5, 1.0),
        'ratio': (0.75, 1.33)
    }
)

---

## 20. ElasticTransform — Эластичные деформации

### Описание и математика

**Elastic Transform (Эластичная деформация)** — это сложная геометрическая аугментация, имитирующая эластичные искажения, как будто изображение напечатано на резиновой поверхности.

**Идея из статьи** "Best Practices for Convolutional Neural Networks Applied to Visual Document Analysis" (Simard et al., 2003):

**Алгоритм:**

1. **Генерация случайных полей смещения**:
   $$\Delta x(i, j) \sim \mathcal{U}(-1, 1)$$
   $$\Delta y(i, j) \sim \mathcal{U}(-1, 1)$$

2. **Применение гауссовского размытия** к полям смещения:
   $$\Delta x_{\text{smooth}} = G_\sigma * \Delta x$$
   $$\Delta y_{\text{smooth}} = G_\sigma * \Delta y$$
   
   где $G_\sigma$ — гауссово ядро с стандартным отклонением $\sigma$.

3. **Масштабирование** смещений параметром $\alpha$ (интенсивность деформации):
   $$\Delta x_{\text{final}} = \alpha \cdot \Delta x_{\text{smooth}}$$
   $$\Delta y_{\text{final}} = \alpha \cdot \Delta y_{\text{smooth}}$$

4. **Применение деформации**:
   $$x_{\text{new}}(i, j) = i + \Delta x_{\text{final}}(i, j)$$
   $$y_{\text{new}}(i, j) = j + \Delta y_{\text{final}}(i, j)$$
   $$I_{\text{result}}(i, j) = I_{\text{old}}(x_{\text{new}}(i, j), y_{\text{new}}(i, j))$$

**Параметры:**
- `alpha`: интенсивность деформации (обычно 0-120)
- `sigma`: параметр размытия полей смещения (обычно 10-20)
- `alpha_affine`: дополнительные аффинные трансформации (опционально)

**Связь параметров:**
- Большой $\alpha$ → сильные искажения
- Большой $\sigma$ → плавные, крупномасштабные искажения
- Малый $\sigma$ → локальные, мелкомасштабные искажения

**Применение:**
- Эффективно для распознавания рукописных символов (MNIST)
- Медицинская сегментация (имитация деформаций органов)
- Повышение робастности к геометрическим искажениям

### Полезные функции OpenCV

- `cv2.GaussianBlur(image, ksize, sigma)` — размытие полей смещения
- `cv2.remap(image, map_x, map_y, interpolation)` — применение деформации
- `np.random.uniform(low, high, size)` — генерация случайных полей

---

In [ ]:
def custom_elastic_transform(image: np.ndarray, 
                             alpha: float = 50,
                             sigma: float = 5,
                             interpolation: int = cv2.INTER_LINEAR,
                             border_mode: int = cv2.BORDER_REFLECT_101) -> np.ndarray:
    """
    Применяет эластичные деформации к изображению.
    
    Args:
        image: Входное RGB изображение
        alpha: Интенсивность деформации
        sigma: Параметр размытия для сглаживания деформации
        interpolation: Метод интерполяции
        border_mode: Режим обработки границ
    
    Returns:
        Деформированное изображение
    """
    # TODO
    
    # Применяем деформацию
    result = cv2.remap(image, map_x, map_y, 
                      interpolation=interpolation,
                      borderMode=border_mode)
    
    return result

# Визуализация сравнения
visualize_comparison(
    image=img_geom_rgb,
    custom_func=custom_elastic_transform,
    albumentations_transform=A.ElasticTransform(
        alpha=100,
        sigma=5,
        alpha_affine=0,
        p=1.0
    ),
    title="ElasticTransform: Сравнение реализаций",
    params={'alpha': 50, 'sigma': 5}
)

---

## 22. Dithering — Дизеринг (подробная теория и реализация)

### Введение и история

**Dithering (дизеринг)** — это техника создания иллюзии большей глубины цвета путем пространственного распределения ограниченной палитры цветов. Исторически использовался для отображения изображений на устройствах с ограниченным количеством цветов (ранние мониторы, печатные машины, факсы).

### Математическая теория

#### Проблема квантования цвета

При уменьшении битовой глубины с 8 бит до $n$ бит возникает **ошибка квантования**:

$$e(x, y) = I(x, y) - Q_n(I(x, y))$$

где:
- $I(x, y) \in [0, 255]$ — исходное значение пикселя
- $Q_n(\cdot)$ — функция квантования до $2^n$ уровней
- $e(x, y)$ — локальная ошибка квантования

**Функция квантования:**
$$Q_n(v) = \left\lfloor \frac{v}{s} \right\rfloor \times s, \quad s = \frac{256}{2^n}$$

Простое квантование создает **banding artifacts** (видимые полосы границ между уровнями).

---

### Метод 1: Floyd-Steinberg Dithering (Error Diffusion)

**Ключевая идея:** распределить ошибку квантования текущего пикселя на соседние еще не обработанные пиксели.

**Алгоритм:**

1. Обрабатываем пиксели **построчно слева направо, сверху вниз**
2. Для текущего пикселя $(x, y)$:
   - Квантуем: $I'(x, y) = Q_n(I(x, y))$
   - Вычисляем ошибку: $e = I(x, y) - I'(x, y)$
   - Распределяем ошибку на соседей

**Паттерн распределения ошибки:**

$$\begin{matrix}
& & \times & \frac{7}{16}e \\
\frac{3}{16}e & \frac{5}{16}e & \frac{1}{16}e &
\end{matrix}$$

где $\times$ — текущий пиксель.

**Формулы распределения:**

$$I(x+1, y) \leftarrow I(x+1, y) + e \cdot \frac{7}{16}$$
$$I(x-1, y+1) \leftarrow I(x-1, y+1) + e \cdot \frac{3}{16}$$
$$I(x, y+1) \leftarrow I(x, y+1) + e \cdot \frac{5}{16}$$
$$I(x+1, y+1) \leftarrow I(x+1, y+1) + e \cdot \frac{1}{16}$$

**Важно:** Сумма коэффициентов = $\frac{7+3+5+1}{16} = 1$ — **вся ошибка распределяется**!

**Свойства:**
- Высокое визуальное качество
- Создает **blue noise** (высокочастотный шум, менее заметный глазу)
- Последовательная обработка (нельзя распараллелить)
- Возможны артефакты "червей" при резких градиентах

---

### Метод 2: Ordered Dithering (Bayer Matrix)

**Ключевая идея:** использовать предопределенную матрицу пороговых значений для квантования.

**Матрица Байера 2×2:**

$$B_2 = \frac{1}{4} \begin{bmatrix}
0 & 2 \\
3 & 1
\end{bmatrix} = \begin{bmatrix}
0.00 & 0.50 \\
0.75 & 0.25
\end{bmatrix}$$

**Матрица Байера 4×4:**

$$B_4 = \frac{1}{16} \begin{bmatrix}
0 & 8 & 2 & 10 \\
12 & 4 & 14 & 6 \\
3 & 11 & 1 & 9 \\
15 & 7 & 13 & 5
\end{bmatrix}$$

**Рекурсивная формула для построения матриц:**

$$B_{2n} = \begin{bmatrix}
4B_n & 4B_n + 2U_n \\
4B_n + 3U_n & 4B_n + U_n
\end{bmatrix}$$

где $U_n$ — матрица единиц размера $n \times n$.

**Применение:**

$$I'(x, y) = Q_n\left(I(x, y) + (B_{ij} - 0.5) \times s\right)$$

где:
- $i = x \mod n, \quad j = y \mod n$ (индексы в матрице Байера)
- $s$ — размер шага квантования
- $(B_{ij} - 0.5)$ центрирует значение вокруг 0

**Свойства:**
- Очень быстрый (параллелизуется)
- Создает видимый регулярный паттерн
- Среднее качество
- Часто используется в печати

---

### Сравнение методов

| Критерий | Floyd-Steinberg | Ordered (Bayer) | Simple Quantization |
|----------|-----------------|-----------------|---------------------|
| **Качество** | ★★★★★ | ★★★☆☆ | ★☆☆☆☆ |
| **Скорость** | ★★☆☆☆ | ★★★★★ | ★★★★★ |
| **Артефакты** | "Черви" (редко) | Видимый паттерн | Сильный banding |
| **Частотный состав** | Blue noise | Регулярная структура | Низкочастотные полосы |
| **Параллелизация** | Нет | Да | Да |

---

### Применение в Computer Vision

**Зачем использовать дизеринг для ML:**

1. **Имитация низкобитных устройств** (старые камеры, дисплеи)
2. **Устойчивость к квантованию** (модели на edge-устройствах)
3. **Подготовка к квантизации весов** (INT8, binary networks)
4. **Художественная стилизация** (ретро-эффект)
5. **Робастность к шуму** квантования

**Частотный анализ:**

Дизеринг переносит ошибку из низких частот (заметные артефакты) в высокие частоты (мелкий шум):

$$\mathcal{F}\{I_{\text{dithered}}\} \approx \mathcal{F}\{I_{\text{original}}\} + \mathcal{F}\{n_{\text{high-freq}}\}$$

где $n_{\text{high-freq}}$ — высокочастотный шум, менее воспринимаемый глазом.

---

### Теоретическое обоснование (почему это работает)

**1. Spatial Averaging (Пространственное усреднение):**

Человеческий глаз усредняет близлежащие пиксели, создавая иллюзию промежуточных цветов:

$$I_{\text{perceived}}(x, y) \approx \frac{1}{|N|} \sum_{(i,j) \in N(x,y)} I(i, j)$$

**2. Energy Preservation (Сохранение энергии):**

Суммарная яркость сохраняется:

$$\sum_{x,y} I_{\text{dithered}}(x,y) \approx \sum_{x,y} I_{\text{original}}(x,y)$$

**3. Blue Noise Property:**

Floyd-Steinberg создает шум, близкий к blue noise (высокочастотный спектр), который:
- Менее заметен для человеческого глаза
- Не создает регулярных паттернов
- Имеет плоскую спектральную плотность на высоких частотах



In [ ]:
def custom_floyd_steinberg_dithering(image: np.ndarray,
                                      num_bits: int = 1) -> np.ndarray:
    """
    Применяет Floyd-Steinberg error diffusion dithering.
    
    Args:
        image: Входное изображение (H, W, C)
        num_bits: Количество бит на канал (1-8)
    
    Returns:
        Дизеренное изображение
    """
    # TODO (со звездочкой)
    return result


def custom_ordered_dithering(image: np.ndarray,
                             num_bits: int = 1,
                             matrix_size: int = 4) -> np.ndarray:
    """
    Применяет ordered dithering с матрицей Байера.
    
    Args:
        image: Входное изображение
        num_bits: Количество бит на канал
        matrix_size: Размер матрицы Байера (2 или 4)
    
    Returns:
        Дизеренное изображение
    """
    # TODO (со звездочкой)
    return result


# Визуализация сравнения обоих методов
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# Оригинал
axes[0, 0].imshow(img_color_rgb)
axes[0, 0].set_title('Original (8 bits)', fontsize=12, weight='bold')
axes[0, 0].axis('off')

# Простое квантование (для сравнения)
for i, bits in enumerate([1, 2, 4], 1):
    step = 256 // (2 ** bits)
    simple = (img_color_rgb // step) * step
    axes[0, i].imshow(simple)
    axes[0, i].set_title(f'Simple Quantization\n({bits} bit = {2**bits} levels)\n⚠️ Banding!', fontsize=10)
    axes[0, i].axis('off')

# Floyd-Steinberg
axes[1, 0].text(0.5, 0.5, 'Floyd-Steinberg\n(Error Diffusion)', 
                ha='center', va='center', fontsize=12, weight='bold',
                transform=axes[1, 0].transAxes)
axes[1, 0].axis('off')

for i, bits in enumerate([1, 2, 4], 1):
    dithered = custom_floyd_steinberg_dithering(img_color_rgb.copy(), num_bits=bits)
    axes[1, i].imshow(dithered)
    axes[1, i].set_title(f'Floyd-Steinberg\n({bits} bit)\n✅ Smooth', fontsize=10)
    axes[1, i].axis('off')

# Ordered (Bayer)
axes[2, 0].text(0.5, 0.5, 'Ordered Dithering\n(Bayer Matrix)', 
                ha='center', va='center', fontsize=12, weight='bold',
                transform=axes[2, 0].transAxes)
axes[2, 0].axis('off')

for i, bits in enumerate([1, 2, 4], 1):
    dithered = custom_ordered_dithering(img_color_rgb.copy(), num_bits=bits, matrix_size=4)
    axes[2, i].imshow(dithered)
    axes[2, i].set_title(f'Ordered/Bayer\n({bits} bit)\n⚠️ Pattern', fontsize=10)
    axes[2, i].axis('off')

plt.tight_layout()
plt.show()

# Дополнительная визуализация: матрица Байера 4x4
bayer_4x4 = np.array([
    [0, 8, 2, 10],
    [12, 4, 14, 6],
    [3, 11, 1, 9],
    [15, 7, 13, 5]
]) / 16.0

fig, ax = plt.subplots(1, 1, figsize=(7, 7))
im = ax.imshow(bayer_4x4, cmap='gray', interpolation='nearest')
ax.set_title('Bayer Matrix 4×4 (Threshold Pattern)', fontsize=14, weight='bold')

# Добавляем значения в ячейки
for i in range(4):
    for j in range(4):
        text = ax.text(j, i, f'{bayer_4x4[i, j]:.3f}',
                      ha="center", va="center", color="red", fontsize=12, weight='bold')

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.grid(True, color='white', linewidth=2)
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()


---

## 📊 Итоги Части 3

### Что вы реализовали:

**Простые цветовые аугментации:**
1. ✅ **RGBShift** — независимый сдвиг RGB каналов
2. ✅ **Downscale** — понижение разрешения с восстановлением
3. ✅ **Equalize** — эквализация гистограммы
4. ✅ **Posterize** — постеризация (уменьшение цветовой глубины)
5. ✅ **CLAHE** — адаптивная локальная эквализация

**Продвинутые цветовые:**
6. ✅ **ColorJitter** — комплексное изменение яркости, контраста, насыщенности и оттенка

**Фильтры и эффекты:**
7. ✅ **Emboss** — эффект тиснения (подчеркивание границ)
8. ✅ **Sharpen** — повышение резкости (Unsharp Masking)

**Размытия:**
9. ✅ **GaussianBlur** — гауссово размытие
10. ✅ **MotionBlur** — размытие движения
11. ✅ **ZoomBlur** — размытие при зумировании камеры

**Шумы и артефакты:**
12. ✅ **ISONoise** — ISO шум камеры
13. ✅ **Spatter** — брызги и пятна (дождь/грязь)
14. ✅ **ChromaticAberration** — хроматическая аберрация оптики
15. ✅ **GlassBlur** — эффект матового стекла

**Продвинутые цветовые преобразования:**
16. ✅ **FancyPCA** — PCA-аугментация цветов (AlexNet style)

**Геометрические трансформации:**
17. ✅ **Rotate** — поворот изображения
18. ✅ **Perspective** — перспективная трансформация (гомография)
19. ✅ **RandomResizedCrop** — случайный кроп с масштабированием
20. ✅ **ElasticTransform** — эластичные деформации

**Сложные методы обработки:**
21. ✅ **Dithering** — дизеринг (Floyd-Steinberg и Bayer Matrix)
